# Aumento de datos

En este notebook se va a aplicar la técnica de ampliación de datos a un conjunto de reseñas de Google Maps separadas en dos ficheros: uno con las reseñas que se van a considerar válidas y el otro con las inválidas. Cada línea es una reseña nueva.

# Aumento de datos tradicional

### Imports

In [1]:
import pandas as pd
from deep_translator import (GoogleTranslator, MyMemoryTranslator)
import copy
import time
import random
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/ibon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ibon/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Direcorio de datos

In [20]:
validReviewsPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidReviewsPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

### Pandas
Se van a pasar los datos a dataframes: uno con las valoraciones validas y otro con las negativas. Cada fila del dataframe será una reseña

In [3]:
def importFromTxtToDF(source):
    with open(source, 'r', encoding="utf-8") as file:
        #Generate a list with all the reviews
        targetList = [line.strip() for line in file]

    targetDF = pd.DataFrame(targetList, columns=['Text'])
    return targetDF

In [4]:
#Read the file with the valid reviews
validReviewsDF = importFromTxtToDF(validReviewsPath)
#Read the file with the invalid reviews
invalidReviewsDF = importFromTxtToDF(invalidReviewsPath)

Se muestran las primeras reseñas válidas

In [5]:
validReviewsDF.head()

,Text
0,"""Tiene fácil acceso para las personas con movi..."
1,"""Espero que hayan mejorais"""
2,"""La estación es antigua, aparte de tener una s..."
3,"""Bien"""
4,"""Bonito comodo"""


Se muestran las primeras reseñas inválidas

In [6]:
invalidReviewsDF.head()

,Text
0,"""He vivido 35 años en el barrio y reconozco qu..."
1,"""localización con muchos bares interesantes"""
2,"""…"""
3,"""Muy rica comida.."""
4,"""Estación del.metro"""


## Medidas de similitud
Para poder comparar frases y seleccionar las mejores para generar los mejores datasets se van a desarrollar las siguientes medidas de similitud.

### Similitud Semántica

A continuación se va a diseñar una función para calcular la similitud semántica entre pares de oraciones. Es decir, se van a calcular los embeddings de oraciones de cada par de frases y se va a usar una métrica de similitud para ver como de parecido es el significado de ambas frases.

Se va a usar una versión de SBERT, llamada MiniLM (Minimal Lenguaje Model), que utiliza una variante más pequeña. Se usa MiniLM de seis capas (L6), que logra una precisón buena con menos recursos.

Este modelo fue entrenado usando un dataset que incluye datos en varios idiomas, entre ellos el español. Consecuentemente, no hay problema al introducir frases en castellano. Es cierto, que obtiene mejores resultados para frases en inglés, ya que se entreno con más datos en este idioma.

MiniLM es un modelo específicamente entrenado para mapear frases y parrafos a un espacio vectorial de 384 dimensiones. Es decir, este modelo permite obtener un embedding de una frase directamente. Usando otros modelos esta tarea no es posible de forma directa, ya que devuelven un embedding para cada palabra del texto.

El método de similitud que se va a usar es la similitud del coseno, por lo que los valores más cercanos a uno indicarán una mayor similitud entre las frases

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

#Given two texts and a model, the semantic similarity of the texts is returned
def getSemanticSimilarity(text1, text2, model):
    #Get the embeddings of the senteces
    embedding1 = model.encode(text1)
    embedding2 = model.encode(text2)

    #Get the cosine similarity of the senteces
    similarity = cosine_similarity([embedding1], [embedding2])

    return similarity[0][0]

### Similitud Léxica
Se va a diseñar una función para calcular la similitud léxica entre pares de oraciones. La similitud léxica mide el grado de coincidencia de palabras o términos entre dos frases o textos, sin tener en cuenta el significado subyacente.

Hay varias formas de realizar este cálculo: similitud del coseno basada en frecuencia de palabras, coeficiente de Jaccard,coeficiente de Dice ...

En este caso, se cree que la mejor opción es usar el coeficiente de Jaccard ya que calcula la similitud en función de la proporción de palabras comunes sobre el total de palabras únicas. Consecuentemente, esto nos permitirá detectar frases con menos coincidencias exactas en palabras.

Cuanto más cercano a uno sea el coeficiente de Jaccard más similares léxicamente serán las frases.

In [3]:
import unicodedata
import re

#Clean up the text removing punctuation, accent marks and convertin everything to lowercase
def cleanText(text):
    text = unicodedata.normalize('NFKD', text.lower()).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [4]:
def getSpanishStopWords():
    determinantes = {"el", "la", "los", "las", "un", "una", "unos", "unas", "este", "esta", "estos", "estas",
                 "ese", "esa", "esos", "esas", "aquel", "aquella", "aquellos", "aquellas", "mi", "mis",
                 "tu", "tus", "su", "sus", "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", 
                 "vuestra", "vuestros", "vuestras", "primer", "primero", "primera", "segundo", "segunda"}

    preposiciones = {"a", "ante", "bajo", "cabe", "con", "contra", "de", "desde", "durante", "en", "entre", 
                 "hacia", "hasta", "mediante", "para", "por", "según", "sin", "sobre", "tras", "versus", "vía"}

    conjunciones = {"y", "e", "ni", "o", "u", "pero", "sino", "sino que", "mas", "aunque", "que", "porque", 
                "como", "cuando", "donde", "mientras", "para que", "a fin de que", "puesto que", "ya que", 
                "si", "siempre que"}
    pronombres = {
        # Pronombres personales
        "yo", "tú", "vos", "él", "ella", "nosotros", "nosotras", 
        "vosotros", "vosotras", "ellos", "ellas", "usted", "ustedes",
        "me", "te", "lo", "la", "nos", "os", "los", "las", "le", "les", "se",
    
        # Pronombres posesivos
        "mío", "mía", "míos", "mías", 
        "tuyo", "tuya", "tuyos", "tuyas", 
        "suyo", "suya", "suyos", "suyas", 
        "nuestro", "nuestra", "nuestros", "nuestras", 
        "vuestro", "vuestra", "vuestros", "vuestras",
    
        # Pronombres demostrativos
        "este", "esta", "estos", "estas", 
        "ese", "esa", "esos", "esas", 
        "aquel", "aquella", "aquellos", "aquellas",
    
        # Pronombres relativos
        "que", "cual", "cuales", "quien", "quienes", 
        "cuyo", "cuya", "cuyos", "cuyas", "donde",
    
        # Pronombres interrogativos y exclamativos
        "qué", "quién", "quiénes", "cuál", "cuáles", 
        "cuánto", "cuánta", "cuántos", "cuántas", 
        "dónde", "cómo", "cuándo",
    
        # Pronombres indefinidos
        "alguien", "algo", "nadie", "nada", "cualquiera", 
        "todos", "todas", "varios", "varias", "muchos", 
        "muchas", "pocos", "pocas", "alguno", "alguna", 
        "algunos", "algunas", "ninguno", "ninguna", 
        "uno", "una", "unos", "unas", "demás"
    }

    #Combine all the words in one set
    spanishStopWords = determinantes | preposiciones | conjunciones | pronombres

    return spanishStopWords

In [5]:
def revomeSpanishStopWords(text):
    spanishStopWords = getSpanishStopWords()

    textWithoutStopWords = [word for word in text.split() if word.lower() not in spanishStopWords]

    return " ".join(textWithoutStopWords)

In [6]:
#Jaccard similarity
def jaccardSimilarity(text1, text2):
    #Get the set of words of each text
    wordsInText1 = set(revomeSpanishStopWords(cleanText(text1)).split())
    wordsInText2 = set(revomeSpanishStopWords(cleanText(text2)).split())

    intersection = len(wordsInText1.intersection(wordsInText2)) 
    union = len(wordsInText1.union(wordsInText2))
    
    if union == 0:
        return 0

    #intersection / union
    return intersection / union

In [7]:
#Given two texts, the Jaccard similarity of those texts is returned
def getLexicalSimilarity(text1, text2):
    return jaccardSimilarity(text1, text2)

## Retrotraducción

El primer método de ampliación de datos que se va a usar va a ser la retrotraducción. Consiste en traducir el texto a un idioma distinto y luego volverlo a traducir al idioma original. 

Este proceso puede genera texto con el mismo significado que el original pero distintas palabras.

In [7]:
def BackTranslation(translatorsList, reviewsDF, targetPath):
    #Generate a data list to store the text that has to be translated
    notTranslatedList = reviewsDF['Text'].tolist()

    #Translate the text as many times as needed
    for translator in translatorsList:
        #Generate a data frame to store the text that has been translated
        translatedList = []
        for elem in notTranslatedList:
            #Translate all the reviews
            try:
                translation = translator.translate(elem)
            except Exception as e: #If the translation fails "" is written
                translation = '""'
            #If an error ocurred translate it to a ""
            if translation == None:
                translation = '""'
                
            #Save the translations in the corresponding list 
            translatedList.append(translation)

            #Wait 0.2 seconds not to collapse the server
            time.sleep(0.2)

        #Prepare to translate again if needed
        notTranslatedList = copy.deepcopy(translatedList) 
       
    #Open the file in which the translations are strored
    translationFile = open(targetPath, 'w', encoding="utf-8")
    #write all the translations
    for elem in translatedList:
        translationFile.write(elem + "\n")
    #Close the file
    translationFile.close()
    
    return pd.DataFrame(translatedList, columns=['Text'])

### Google Translator

Primero se va a traducir del castellano al ingles y luego del inglés al castellano

In [8]:
validPath = '1. Back Translation\\1. Google Translator\\ValidReviewsTranslationsEsEnEnEs.txt'
invalidPath = '1. Back Translation\\1. Google Translator\\InvalidReviewsTranslationsEsEnEnEs.txt'

firstTranslator = GoogleTranslator(source = 'es', target = 'en')
secondTranslator = GoogleTranslator(source='en', target='es')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsGoogleEsEnEnEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsGoogleEsEnEnESDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

A continuación se va a traducir del castellano al japonés y del japonés al castellano

In [9]:
validPath = '1. Back Translation\\1. Google Translator\\ValidReviewsTranslationsEsJaJaEs.txt'
invalidPath = '1. Back Translation\\1. Google Translator\\InvalidReviewsTranslationsEsJaJaEs.txt'

firstTranslator = GoogleTranslator(source = 'es', target = 'ja')
secondTranslator = GoogleTranslator(source='ja', target='es')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsGoogleEsJaJaEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsGoogleEsJaJaEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

Por último se va a implementar una cadena de traducciones más larga: castellano a frances, frances a japones, japones a ruso y ruso a catellano.

In [10]:
validPath = '1. Back Translation\\1. Google Translator\\ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'
invalidPath = '1. Back Translation\\1. Google Translator\\InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'

firstTranslator = GoogleTranslator(source = 'es', target = 'fr')
secondTranslator = GoogleTranslator(source='fr', target='ja')
thirdTranslator = GoogleTranslator(source='ja', target='ru')
fourthTranslator = GoogleTranslator(source='ru', target='es')

translatorList = [firstTranslator, secondTranslator, thirdTranslator, fourthTranslator]

validSpanishReviewsGoogleEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsGoogleEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

### MyMemory Translator

Se van a realizar las mismas traducciones pero usando otro traductor

Castellano -> Inglés -> Castellano

In [11]:
validPath = '1. Back Translation\\2. MyMemory Translator\\ValidReviewsTranslationsEsEnEnEs.txt'
invalidPath = '1. Back Translation\\2. MyMemory Translator\\InvalidReviewsTranslationsEsEnEnEs.txt'

firstTranslator = MyMemoryTranslator(source = 'spanish', target = 'english')
secondTranslator = MyMemoryTranslator(source='english', target='spanish')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsMyMemoryEsEnEnEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsMyMemoryEsEnEnEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

Castellano -> Japonés -> Castellano

In [12]:
validPath = '1. Back Translation\\2. MyMemory Translator\\ValidReviewsTranslationsEsJaJaEs.txt'
invalidPath = '1. Back Translation\\2. MyMemory Translator\\InvalidReviewsTranslationsEsJaJaEs.txt'

firstTranslator = MyMemoryTranslator(source = 'spanish', target = 'japanese')
secondTranslator = MyMemoryTranslator(source='japanese', target='spanish')

translatorList = [firstTranslator, secondTranslator]

validSpanishReviewsMyMemoryEsJaJaEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsMyMemoryEsJaJaEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

Castellano -> Francés -> Japonés -> Ruso -> Castellano

In [13]:
validPath = '1. Back Translation\\2. MyMemory Translator\\ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'
invalidPath = '1. Back Translation\\2. MyMemory Translator\\InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'

firstTranslator = MyMemoryTranslator(source = 'spanish', target = 'french')
secondTranslator = MyMemoryTranslator(source='french', target='japanese')
thirdTranslator = MyMemoryTranslator(source='japanese', target='russian')
fourthTranslator = MyMemoryTranslator(source='russian', target='spanish')

translatorList = [firstTranslator, secondTranslator, thirdTranslator, fourthTranslator]

validSpanishReviewsMyMemoryEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, validReviewsDF, validPath)
invalidSpanishReviewsMyMemoryEsFrFrJaJaRuRuEsDF = BackTranslation(translatorList, invalidReviewsDF, invalidPath)

## Análisis de la retrotraducción y selección de los datos

Dado que el código correspondiente a la traducción llevó largo rato y se dejo a la noche ejecutando, se vuelven a importar los datos a dataframes:

#### MyMemory genera errores en la traducción debido a problemas de conexión con el servidor. Consecuentemente, se procede a anilizar únicamente los datos generados por el traductor de Google

In [8]:
def importFromTxtToList(source):
    with open(source, 'r', encoding="utf-8") as file:
        #Generate a list with all the reviews
        targetList = [line.strip() for line in file]
    return targetList

Frases originales

In [8]:
validPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validPath)
invalidOriginal = importFromTxtToList(invalidPath) 

Se importan las frases traducidas de: Castellano -> Inglés -> Castellano

In [9]:
validPath = '1. Back Translation/1. Google Translator/ValidReviewsTranslationsEsEnEnEs.txt'
invalidPath = '1. Back Translation/1. Google Translator/InvalidReviewsTranslationsEsEnEnEs.txt'

validEsEnEnEsTraductionList = importFromTxtToList(validPath)
invalidEsEnEnEsTraductionList = importFromTxtToList(invalidPath) 

Castellano -> Japonés -> Castellano

In [10]:
validPath = '1. Back Translation/1. Google Translator/ValidReviewsTranslationsEsJaJaEs.txt'
invalidPath = '1. Back Translation/1. Google Translator/InvalidReviewsTranslationsEsJaJaEs.txt'

validEsJaJaEsTraductionList = importFromTxtToList(validPath)
invalidEsJaJaEsTraductionList = importFromTxtToList(invalidPath)

Castellano -> Francés -> Japonés -> Ruso -> Castellano

In [11]:
validPath = '1. Back Translation/1. Google Translator/ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'
invalidPath = '1. Back Translation/1. Google Translator/InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt'

validEsFrFrJaJaRuRuEsList = importFromTxtToList(validPath)
invalidEsFrFrJaJaRuRuEsList = importFromTxtToList(invalidPath)

Para cada frase del conjunto de datos original (las frases con las reseñas válidas e inválidas), se va a calcular la similitud semántica (usando a un modelo basado en SBERT, conocido como MiniLM, que es más ligero y rápido consiguiendo resultados bastante certeros) y léxica con sus tres correspondientes frases generadas mediante el métodod de retrotraducción. Se van a seleccionar las frases que tengan mayor similitud semántica y menor similitud léxica y se van a guardar en un fichero para su posterior uso.

Se van a generar dos ficheros: un csv con dos elementos por fila (la frase original y la retrotraducción escogida) y el otro con solo las retrotraducciones seleccionadas.

In [10]:
from sentence_transformers import SentenceTransformer

#originalDataList: list of texts representing the original dataset
#allAugmentedDataList: list of list of texts representing the aumented data 
#(allAugmentedDataList = [augmentedDataList1, ... ,augmentedDataListN], where augmentedDataList = [augmentedData1, ..., augmentedDataM])
#pathWithOriginal: path of the csv with two columns (the original text and the best augmented text)
#pathAugmentedData: path of the file with only the augmented data (without the original text)
def processAugmentation(originalDataList, allAugmentedDataList, pathWithOriginal, pathAugmentedData):
    #Select the model for the semantic similarity
    model = SentenceTransformer('all-MiniLM-L6-v2')

    #Open the files in which the augmented data will be strored
    withOriginalFile = open(pathWithOriginal, "w", encoding="utf-8")
    augmentedDataFile = open(pathAugmentedData, "w", encoding="utf-8")

    #Write the titles of the csv
    withOriginalFile.write("OriginalText,AugmentedText,SemanticSimilarity,LexicalSimilarity\n")
    
    resul = []
    #Analize every phrase in the original data
    for i, originalText in enumerate(originalDataList):
        allAugmentedDataInfoDict = {}
        bestIdx = 1
        
        #Analize every traduction
        for j, augmentedDataList in enumerate(allAugmentedDataList):
            #Compute the similarities of the corresponding traduction
            semanticSimilarity = getSemanticSimilarity(originalText, augmentedDataList[i], model)
            lexicalSimilarity = getLexicalSimilarity(originalText, augmentedDataList[i])

            #Save the traduction and the similarities in a dictionary
            allAugmentedDataInfoDict.update({
                f"augmented{j + 1}": augmentedDataList[i],
                f"semanticSimilarity{j + 1}": semanticSimilarity,
                f"lexicalSimilarity{j + 1}": lexicalSimilarity
            })

            #Get the index of the traduction with greater semantic similarity and less lexical similarity
            bestIdx = max(bestIdx, j + 1,
                key = lambda k: (allAugmentedDataInfoDict[f"semanticSimilarity{k}"] - allAugmentedDataInfoDict[f"lexicalSimilarity{k}"])
            )

        #Select the information of the best augmentation
        info = {
            "originalText": originalText,
            "bestAugmentation": allAugmentedDataInfoDict[f"augmented{bestIdx}"],
            "bestAugmentedDataSemanticSimilarity": allAugmentedDataInfoDict[f"semanticSimilarity{bestIdx}"],
            "bestAugmentedDataLexicalSimiliratity": allAugmentedDataInfoDict[f"lexicalSimilarity{bestIdx}"]
        }
        info.update(allAugmentedDataInfoDict)

        #Save the information
        resul.append(info)

        #Write the information in the files
        withOriginalFile.write(originalText + "," + allAugmentedDataInfoDict[f"augmented{bestIdx}"] + "," + str(allAugmentedDataInfoDict[f"semanticSimilarity{bestIdx}"]) + "," + str(allAugmentedDataInfoDict[f"lexicalSimilarity{bestIdx}"]) + "\n")
        #If the text is not empty write it on  the file
        if allAugmentedDataInfoDict[f"augmented{bestIdx}"]  != '""':
            augmentedDataFile.write(allAugmentedDataInfoDict[f"augmented{bestIdx}"] + "\n")

    #Close the files
    withOriginalFile.close()
    augmentedDataFile.close()
    
    return resul

In [18]:
validWithOriginalPath = '1. Back Translation/3. Augmented Data/ValidBackTranslationWithOriginal.csv'
validAugmentedPath = '1. Back Translation/3. Augmented Data/ValidBackTranslationData.txt'
invalidWithOriginalPath = '1. Back Translation/3. Augmented Data/InvalidBackTranslationWithOriginal.csv'
invalidAugmentedPath = '1. Back Translation/3. Augmented Data/InvalidBackTranslationData.txt'

infoValid = processAugmentation(validOriginal, [validEsEnEnEsTraductionList, validEsJaJaEsTraductionList, validEsFrFrJaJaRuRuEsList], validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, [invalidEsEnEnEsTraductionList, invalidEsJaJaEsTraductionList, invalidEsFrFrJaJaRuRuEsList], invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [19]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...",“Es de fácil acceso para personas con discapac...,0.854630,0.450000,“Tiene fácil acceso para personas con movilida...,0.978443,0.812500,“Es de fácil acceso para personas con movilida...,0.925599,0.611111,“Es de fácil acceso para personas con discapac...,0.854630,0.450000
1,"""Espero que hayan mejorais""","""Espero que hayas mejorado""",0.916635,0.200000,"""Espero que hayas mejorado""",0.916635,0.200000,"""Espero que las cosas estén mejorando"".",0.785516,0.166667,"""Espero que la situación esté mejorando"".",0.736162,0.200000
2,"""La estación es antigua, aparte de tener una s...",“Además de que esta estación es antigua y tien...,0.932238,0.393939,"“La estación es antigua, además de tener una ú...",0.883212,0.592593,“La estación es antigua e inaccesible para per...,0.932935,0.419355,“Además de que esta estación es antigua y tien...,0.932238,0.393939
3,"""Bien""","""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000,"""bien""",1.000000,1.000000,"""bien""",1.000000,1.000000
4,"""Bonito comodo""","""Maravilloso confort""",0.415461,0.000000,"""Bonito y cómodo""",0.955331,1.000000,"""Maravilloso confort""",0.415461,0.000000,“Muy conveniente”,0.229466,0.000000


In [20]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...",“Vivo en esta zona desde hace 35 años y recono...,0.695063,0.322581,“Llevo 35 años viviendo en el barrio y reconoz...,0.949282,0.629630,"""He vivido en esta zona durante 35 años y reco...",0.818717,0.448276,“Vivo en esta zona desde hace 35 años y recono...,0.695063,0.322581
1,"""localización con muchos bares interesantes""","""Ubicación con muchos bares interesantes""",0.793883,0.500000,"""Ubicación con muchos bares interesantes""",0.793883,0.500000,"""Ubicación con muchos bares interesantes""",0.793883,0.500000,“Un lugar con muchos bares interesantes”,0.685196,0.500000
2,"""…""","""""",0.905728,0.000000,"""""",0.905728,0.000000,"""...""",0.838349,0.000000,"""""",0.905728,0.000000
3,"""Muy rica comida..""","""Es una comida muy deliciosa"".",0.644577,0.400000,"""Comida muy deliciosa..""",0.677902,0.500000,"""Es una comida muy deliciosa"".",0.644577,0.400000,“Comida muy sabrosa.”,0.706546,0.500000
4,"""Estación del.metro""","""Estación de metro""",0.963894,0.333333,"""Estación de metro""",0.963894,0.333333,"""estación de metro""",0.963894,0.333333,"""estación de metro""",0.963894,0.333333


Como se puede ver, el método de retrotraducción genera frases con una similitud semántica muy parecida pero con gran variavilidad en la similitud léxica. Por lo tanto, se consiguen frases con distinto vocabulario pero mismo significado.

## Reemplazo por sinónimos

Este método consiste en elejir aleatoriamente n palabras del texto que no sean palabras vacías, y reemplazar cada una de estas palabras por uno de sus sinónimos elegido al azar.

Esta función, dada una palabra devuelve, si existe, un sinónimo.

In [11]:
#Swaps the word given by its synonym
def swapSynonym(word):
    #gets all synonyms from the word given
    synset = wordnet.synsets(word, lang='spa')
    if synset:
        #if the word has one or more synonym we swap it
        synset = wordnet.synsets(word, lang='spa')[0]
        synonymsList = synset.lemma_names('spa') 
        cleanList = [synonym.replace('_', ' ').strip() for synonym in synonymsList]
        #filter to make sure its a diferent word
        differentList = [s for s in cleanList if s.lower() != word.lower()]
        #choose a random synonym if the word has one
        if differentList:
            chosen = random.choice(differentList)
            return chosen
        else:
            return word
    else:
        return word

Dado un texto, cambia con un prob% de probabilidad las palabras no vacías por un sinónimo

In [12]:
def swapBySynonymLine(line, prob):
    # Split the line into individual words
    words = line.split();
    newWords = []

    #Get the spanish stop words
    spanishStopWords = getSpanishStopWords()

    #Analyze all the words in the given text
    for word in words:
        # Check if the word is not a stop word
        if word not in spanishStopWords: 
            # With prob probability, replace the word with a synonym
            if random.random() <= prob:
                newWord = swapSynonym(word)
            else: 
                newWord = word
            newWords.append(newWord)
        else:
            newWords.append(word)
    # Join the words back into a single line and return it
    return ' '.join(newWords)

Dado una lista de textos y una ruta, aplica el métododo de sustitución por sinónimos a todos los elementos de la lista y los alamacena en la ruta proporcionada.

In [13]:
#synonym replacement method
def synonymReplacement(textList, prob, targetPath):
    #Open the file
    targetFile = open(targetPath, "w", encoding = "utf-8")
    
    newList = []
    for line in textList:
        newLine = swapBySynonymLine(line, prob)
        newList.append(newLine)
        targetFile.write(newLine + "\n")

    #Close the file
    targetFile.close()
    
    return newList

In [14]:
def importFromTxtToList(source):
    with open(source, 'r', encoding="utf-8") as file:
        #Generate a list with all the reviews
        targetList = [line.strip() for line in file]
    return targetList

Se va a aplicar el método de reemplazo por sinónimos tres veces para generar tres conjuntos de datos diferentes. Además, la probabilidad de sustitución por sinónimo va a aumentar en cada conjunto de datos: inicialmente 0.25, después 0.45 y finalmente 0.65.

In [14]:
#Import the original data
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

#Apply the synonym replacement 3 times to both datasets
for i in range(3):
    validPath = f"3. Synonym Replacement/1. All Augmented Data/validSynonymsReviews{i + 1}.txt"
    invalidPath = f"3. Synonym Replacement/1. All Augmented Data/invalidSynonymsReviews{i + 1}.txt"

    prob = 0.25 + 2 * i / 10
    
    synonymReplacement(validOriginal, prob, validPath)
    synonymReplacement(invalidOriginal, prob, invalidPath)

## Análisis del reemplazo por sinónimos y selección de los datos

Importar los datos originales

In [12]:
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

Importar los tres datasets generados en el reemplazo por sinónimos

In [13]:
#import all the data 
validSynonymReplacementList = []
invalidSynonymReplacementList = []
for i in range(3):
    validPath = f"3. Synonym Replacement/1. All Augmented Data/validSynonymsReviews{i + 1}.txt"
    invalidPath = f"3. Synonym Replacement/1. All Augmented Data/invalidSynonymsReviews{i + 1}.txt"

    validSynonymReplacementList.append(importFromTxtToList(validPath))
    invalidSynonymReplacementList.append(importFromTxtToList(invalidPath))

Realizar el análisis y la selección de los datos

In [14]:
validWithOriginalPath = '3. Synonym Replacement/2. Augmented Data/ ValidSynonymReplacementWithOriginal.csv'
validAugmentedPath = '3. Synonym Replacement/2. Augmented Data/ValidSynonymReplacementData.txt'
invalidWithOriginalPath = '3. Synonym Replacement/2. Augmented Data/InvalidSynonymReplacementWithOriginal.csv'
invalidAugmentedPath = '3. Synonym Replacement/2. Augmented Data/InvalidSynonymReplacementData.txt'

infoValid = processAugmentation(validOriginal, validSynonymReplacementList, validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, invalidSynonymReplacementList, invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [15]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...","""Tiene fácil entrada para las personas con mov...",0.989551,0.875000,"""Tiene fácil acceso para las personas con movi...",1.000000,1.000000,"""Tiene fácil entrada para las personas con mov...",0.989551,0.875000,"""Tiene fácil acceso para las personas con movi...",0.988377,0.875000
1,"""Espero que hayan mejorais""","""Espero que hayan mejorais""",1.000000,1.000000,"""Espero que hayan mejorais""",1.000000,1.000000,"""Espero que hayan mejorais""",1.000000,1.000000,"""Espero que hayan mejorais""",1.000000,1.000000
2,"""La estación es antigua, aparte de tener una s...","""La estación es antigua, a un lado de parir un...",0.805127,0.516129,"""La estación es antigua, aparte de tener una s...",0.949019,0.833333,"""La estación es antigua, a un lado de tener un...",0.952585,0.692308,"""La estación es antigua, a un lado de parir un...",0.805127,0.516129
3,"""Bien""","""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000
4,"""Bonito comodo""","""Bonito comodo""",1.000000,1.000000,"""Bonito comodo""",1.000000,1.000000,"""Bonito comodo""",1.000000,1.000000,"""Bonito comodo""",1.000000,1.000000


In [16]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...","""He vivido 35 largo tiempo en el barrio y reco...",0.920601,0.703704,"""He vivido 35 largo tiempo en el barrio y reco...",0.926218,0.769231,"""He vivido 35 largo tiempo en el barrio y reco...",0.920601,0.703704,"""He vivido 35 mucho tiempo en el barrio y reco...",0.986137,0.8
1,"""localización con muchos bares interesantes""","""localización con muchos bares interesantes""",1.000000,1.000000,"""localización con muchos bares interesantes""",1.000000,1.000000,"""localización con muchos bares interesantes""",1.000000,1.000000,"""localización con muchos bares interesantes""",1.000000,1.0
2,"""…""","""…""",1.000000,0.000000,"""…""",1.000000,0.000000,"""…""",1.000000,0.000000,"""…""",1.000000,0.0
3,"""Muy rica comida..""","""Muy rica comida..""",1.000000,1.000000,"""Muy rica comida..""",1.000000,1.000000,"""Muy rica comida..""",1.000000,1.000000,"""Muy rica comida..""",1.000000,1.0
4,"""Estación del.metro""","""Estación del.metro""",1.000000,1.000000,"""Estación del.metro""",1.000000,1.000000,"""Estación del.metro""",1.000000,1.000000,"""Estación del.metro""",1.000000,1.0


A diferencia de la retrotraducción, este método no genera variabilidad léxica. No altera el significado de la frase, pero tampoco altera las palabras que la componen. Creemos que esto se debe a la reducida capacidad de la librelia nltk para palabras en castellano.

## Inserción aleatoria

Este método consiste en encontrar un sinónimo aleatorio de una palabra aleatoria en la oración que no sea una palabra vacía e insertar ese sinónimo en una posición aleatoria de la oración.

Función para calcular el número de modificaciones (inserciones, eliminaciones, sustituciones ...) de un texto dado dependiendo de su longitud.

In [11]:
#Function to calculate the number of insertions or replacements or deletions on a given text depending on its length
def calculateModifications(text):
    return max(1, int(len(text.split()) * 0.1))

Función para añadir dobles comillas al inicio y al final del texto

In [12]:
#Add quotes to the given text
def addQuotes(text):
    return f'"{text}"'

Función para eliminar las dobles comillas del inicio y el final de un texto

In [13]:
#Remove quotes from the given text
def removeQuotes(text):
    if text.startswith('"') and text.endswith('"'):
        return text[1:-1]
    return text

Función que realiza la inserción aleatoria de un texto

In [18]:
#Function that executes the random insertion on a given text
def wordInsertion(text):
    #Split the text
    wordsList = text.split()   
    
    # Clean the line of text and remove extra spaces or special characters
    cleanTextStr = cleanText(text)
    
    # Remove Spanish stop words and split the result into words
    withoutStopWordsList = revomeSpanishStopWords(cleanTextStr).split() 
    
    # Proceed only if there are words left after removing stop words
    if withoutStopWordsList:
        # Determine the number of insertions based on line length
        for i in range(calculateModifications(text)):
            # Choose a random important word from the list without stop words
            chosen = random.choice(withoutStopWordsList)    
            # Get a synonym of the chosen word
            synonym = swapSynonym(chosen) 
            # Choose a random position to insert the synonym
            pos = random.randint(0, len(wordsList))    
            # Insert the synonym at the chosen position, removing any extra spaces
            wordsList.insert(pos, synonym.strip())   
            
    # Return the modified line with quotation marks around it
    return ' '.join(wordsList)

Función que realiza la inserción aleatoria a todos los elementos de una lista de textos

In [19]:
#Function that executes the random insertion to all the elements of a list of texts and strores it in a file
def randomInsertion(textList, targetPath):
    #Open the file
    targetFile = open(targetPath, "w", encoding = "utf-8")
    
    newList = []
    for text in textList:
        newLine = wordInsertion(text)
        newList.append(newLine)
        targetFile.write(newLine + "\n")

    #Close the file
    targetFile.close()
    
    return newList

Se va a realizar este proceso tres veces para generar más textos distintos de los que se posteriormente se elegirán los mejores.

In [17]:
#Import the original data
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

#Apply the synonym replacement 3 times to both datasets
for i in range(3):
    validPath = f"4. Random Insertion/1. All Augmented Data/validRandomInsertionReviews{i + 1}.txt"
    invalidPath = f"4. Random Insertion/1. All Augmented Data/invalidRandomInsertionReviews{i + 1}.txt"

    
    randomInsertion(validOriginal, validPath)
    randomInsertion(invalidOriginal, invalidPath)

## Análisis de la inserción aleatoria y selección de los datos

Importar los datos originales

In [11]:
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

Importar los datasets generados en la inserción aleatoria

In [12]:
#import all the data 
validRandomInsertionList = []
invalidRandomInsertionList = []
for i in range(3):
    validPath = f"4. Random Insertion/1. All Augmented Data/validRandomInsertionReviews{i + 1}.txt"
    invalidPath = f"4. Random Insertion/1. All Augmented Data/invalidRandomInsertionReviews{i + 1}.txt"

    validRandomInsertionList.append(importFromTxtToList(validPath))
    invalidRandomInsertionList.append(importFromTxtToList(invalidPath))

Realizar el análisis y la selección de los datos

In [13]:
validWithOriginalPath = '4. Random Insertion/2. Augmented Data/ ValidRandomInsertionWithOriginal.csv'
validAugmentedPath = '4. Random Insertion/2. Augmented Data/ValidRandomInsertionData.txt'
invalidWithOriginalPath = '4. Random Insertion/2. Augmented Data/InvalidRandomInsertionWithOriginal.csv'
invalidAugmentedPath = '4. Random Insertion/2. Augmented Data/InvalidRandomInsertionData.txt'

infoValid = processAugmentation(validOriginal, validRandomInsertionList, validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, invalidRandomInsertionList, invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [14]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...","""Tiene fácil acceso para las personas con movi...",0.993096,0.937500,"""Tiene julian fácil acceso para las personas c...",0.958814,1.000000,"""Tiene fácil acceso otro para las personas con...",0.965412,1.000000,"""Tiene fácil acceso para las personas con movi...",0.993096,0.937500
1,"""Espero que hayan mejorais""","""Espero que hayan mejorais"" hayan",0.988550,1.000000,"espero ""Espero que hayan mejorais""",0.977621,1.000000,"""Espero que hayan mejorais"" hayan",0.988550,1.000000,"""Espero hayan que hayan mejorais""",0.987335,1.000000
2,"""La estación es antigua, aparte de tener una s...","""La estación es antigua, aparte de tener una s...",0.986128,0.880000,"""La estación es antigua, aparte de tener una s...",0.986128,0.880000,"""La estación es antigua, aparte de tener una s...",0.978366,0.916667,"""La estación es antigua, aparte de tener una a...",0.968685,0.916667
3,"""Bien""","""Bien"" bien",0.956442,1.000000,"""Bien"" bien",0.956442,1.000000,"""Bien"" bien",0.956442,1.000000,"""Bien"" bien",0.956442,1.000000
4,"""Bonito comodo""","precioso ""Bonito comodo""",0.932838,0.666667,"precioso ""Bonito comodo""",0.932838,0.666667,"""Bonito comodo comodo""",0.973187,1.000000,"""Bonito comodo"" comodo",0.978755,1.000000


In [15]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...","""He vivido dolor 35 años en joven el barrio y ...",0.980646,0.88,"""He vivido dolor 35 años en joven el barrio y ...",0.980646,0.88,"""He zonas joven vivido 35 años en el barrio y ...",0.946283,1.00,"""He vivido 35 años en el zonas barrio desapare...",0.967516,1.00
1,"""localización con muchos bares interesantes""","""localización con bares muchos bares interesan...",0.977484,1.00,"""localización bares con muchos bares interesan...",0.974223,1.00,"""localización con muchos bares bares interesan...",0.966685,1.00,"""localización con bares muchos bares interesan...",0.977484,1.00
2,"""…""","""…""",1.000000,0.00,"""…""",1.000000,0.00,"""…""",1.000000,0.00,"""…""",1.000000,0.00
3,"""Muy rica comida..""","""Muy rica comida.."" sustancialmente",0.935908,0.75,"""Muy rica sustancialmente comida..""",0.878053,0.75,"""Muy rica comida.."" sustancialmente",0.935908,0.75,"alimento ""Muy rica comida..""",0.929850,0.75
4,"""Estación del.metro""","""Estación estacion del.metro""",0.980573,1.00,"estacion ""Estación del.metro""",0.972385,1.00,"""Estación del.metro"" estacion",0.978640,1.00,"""Estación estacion del.metro""",0.980573,1.00


Como se puede ver, da mejores resultados que el anterior método, pero aun asi hay muy poca variación tanto en la similitud semántica como en la léxica (el mayor de los problemas)

## Intercambio aleatorio

Este método consiste en cambiar n veces dos palabras aleatoriamente en un texto.

Esta función va a coger dos índices aleatorios y distintos de una lista.

In [20]:
#Get two different indexes of a list
def getRandomIndexes(wordsList):
    
    # Generate a random index within the range of the words list
    pos1 = random.randint(0, len(wordsList) - 1)
    
    # Keep generating a new index until it is not equal to the first index
    pos2 = random.randint(0, len(wordsList) - 1)
    while pos1 == pos2:
        pos2 = random.randint(0, len(wordsList) - 1)   

    #Return both indexes
    return pos1, pos2

Esta funcion selecciona dos palabras no vacías del texto original y las intercambia. Repite este proceso tantas veces como la función alculateModifications(text) lo indique.

In [21]:
#Function that selects two non stop words of a text and swaps them. Does this alculateModifications(text) times
def wordSwap(text):
    #Remove the quotation marks
    text = removeQuotes(text)

    #Get the list of words of the text
    wordsList = text.split()
    
    #Remove the spanish stop words from the text
    withoutStopWordsTextList = revomeSpanishStopWords(text).split()
    
    # Check if there are more than one none stop words to perform swapping
    if len(withoutStopWordsTextList) > 1:
        # Loop for the number of modifications calculated for the text
        for i in range(calculateModifications(text)):
            # Get two diferent random indexes
            pos1 , pos2 = getRandomIndexes(withoutStopWordsTextList)

            #Update the indexes to match the original text
            pos1 = wordsList.index(withoutStopWordsTextList[pos1])
            pos2 = wordsList.index(withoutStopWordsTextList[pos2])
            
            # Swap the words at the two random positions
            aux = wordsList[pos1]
            wordsList[pos1] = wordsList[pos2]
            wordsList[pos2] = aux

    
    # Return the modified line with quotes added
    return addQuotes(' '.join(wordsList))

Función que realiza el intercambio aleatorio a todos los elementos de una lista de textos.

In [22]:
#Random swap
def randomSwap(textList, targetPath):
    #Open the file
    targetFile = open(targetPath, "w", encoding = "utf-8")

    newList = []
    for text in textList:
        newText = wordSwap(text)
        newList.append(newText)
        targetFile.write(newText + "\n")

    #Close the file
    targetFile.close()
    
    return newList

Se va a realizar este proceso tres veces para generar más textos distintos de los que se posteriormente se elegirán los mejores.

In [15]:
#Import the original data
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

#Apply the synonym replacement 3 times to both datasets
for i in range(3):
    validPath = f"5. Random Swap/1. All Augmented Data/validRandomSwapReviews{i + 1}.txt"
    invalidPath = f"5. Random Swap/1. All Augmented Data/invalidRandomSwapReviews{i + 1}.txt"

    
    randomSwap(validOriginal, validPath)
    randomSwap(invalidOriginal, invalidPath)

## Análisis del intercambio aleatorio y selección de los datos

Importar los datos originales.

In [11]:
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

Importar los datasets generados en el intercambio aleatorio.

In [12]:
#import all the data 
validRandomSwapList = []
invalidRandomSwapList = []
for i in range(3):
    validPath = f"5. Random Swap/1. All Augmented Data/validRandomSwapReviews{i + 1}.txt"
    invalidPath = f"5. Random Swap/1. All Augmented Data/invalidRandomSwapReviews{i + 1}.txt"

    validRandomSwapList.append(importFromTxtToList(validPath))
    invalidRandomSwapList.append(importFromTxtToList(invalidPath))

Realizar el análisis y la selección de los datos

In [13]:
validWithOriginalPath = '5. Random Swap/2. Augmented Data/ ValidRandomSwapWithOriginal.csv'
validAugmentedPath = '5. Random Swap/2. Augmented Data/ValidRandomSwapData.txt'
invalidWithOriginalPath = '5. Random Swap/2. Augmented Data/InvalidRandomSwapWithOriginal.csv'
invalidAugmentedPath = '5. Random Swap/2. Augmented Data/InvalidRandomSwapData.txt'

infoValid = processAugmentation(validOriginal, validRandomSwapList, validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, invalidRandomSwapList, invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [14]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...","""acceso fácil Tiene para las personas con movi...",0.993650,1.0,"""Tiene fácil acceso para las personas con movi...",0.986013,1.0,"""Tiene fácil acceso para las personas con movi...",0.989630,1.0,"""acceso fácil Tiene para las personas con movi...",0.993650,1.0
1,"""Espero que hayan mejorais""","""Espero que mejorais hayan""",0.992640,1.0,"""hayan que Espero mejorais""",0.989534,1.0,"""Espero que mejorais hayan""",0.992640,1.0,"""mejorais que hayan Espero""",0.991634,1.0
2,"""La estación es antigua, aparte de tener una s...","""La estación es antigua, aparte de tener una n...",0.995518,1.0,"""La estación es antigua, aparte de tener una n...",0.995518,1.0,"""La estación es antigua, aparte de habilitada ...",0.993437,1.0,"""La estación es antigua, pasillo de tener una ...",0.982857,1.0
3,"""Bien""","""Bien""",1.000000,1.0,"""Bien""",1.000000,1.0,"""Bien""",1.000000,1.0,"""Bien""",1.000000,1.0
4,"""Bonito comodo""","""comodo Bonito""",0.988781,1.0,"""comodo Bonito""",0.988781,1.0,"""comodo Bonito""",0.988781,1.0,"""comodo Bonito""",0.988781,1.0


In [15]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...","""He gente 35 años en el ha y reconozco que el ...",0.986286,1.0,"""He gente 35 años en el ha y reconozco que el ...",0.986286,1.0,"""barrio vivido 35 años en el He y reconozco qu...",0.976794,1.0,"""He zonas mejoró años en el barrio y reconozco...",0.970291,1.0
1,"""localización con muchos bares interesantes""","""interesantes con muchos bares localización""",0.991607,1.0,"""interesantes con muchos bares localización""",0.991607,1.0,"""bares con muchos localización interesantes""",0.985325,1.0,"""interesantes con muchos bares localización""",0.991607,1.0
2,"""…""","""…""",1.000000,0.0,"""…""",1.000000,0.0,"""…""",1.000000,0.0,"""…""",1.000000,0.0
3,"""Muy rica comida..""","""rica Muy comida..""",0.989455,1.0,"""Muy comida.. rica""",0.980956,1.0,"""rica Muy comida..""",0.989455,1.0,"""comida.. rica Muy""",0.964634,1.0
4,"""Estación del.metro""","""del.metro Estación""",0.988279,1.0,"""del.metro Estación""",0.988279,1.0,"""del.metro Estación""",0.988279,1.0,"""del.metro Estación""",0.988279,1.0


En este caso, la similitud semántica varía ligeramente, pero la similitud léxica permanece intacta, ya que no se introducen nuevas palabras ni se eliminan palabras existentes.

## Eliminación aleatoria
Este método consiste en eliminar una palabra elegida aleatoriamente en el texto que no sea una palabra vacía.

Función que escoge una palabra no vacía de un texto y la elimina

In [23]:
def deleteWord(text):
    #Remove the quotation marks
    text = removeQuotes(text)

    #Get the list of words of the text
    wordsList = text.split()

    #Remove the spanish stop words from the text
    withoutStopWordsTextList = revomeSpanishStopWords(text).split()
    
    # Calculate and performs the number of deletions based on the line's content
    for i in range(calculateModifications(text)):
        # Check if there are more than one word to delete from
        if len(withoutStopWordsTextList) > 1:
            # Generate a random index to select a word for deletion
            pos = random.randint(0, len(withoutStopWordsTextList) - 1)
    
            #Update the index to the original list (with stop words)
            indx = wordsList.index(withoutStopWordsTextList[pos])
    
            #Remove the chosen word
            wordsList.pop(indx)
            withoutStopWordsTextList.pop(pos)
            
    # Join the remaining words into a string, add quotes, and return the result
    return addQuotes(' '.join(wordsList))

Función que realiza la eliminación aleatoria a todos los elementos de una lista de textos.

In [24]:
def randomDeletion(textList, targetPath):
    #Open the file
    targetFile = open(targetPath, "w", encoding = 'utf-8')
    
    newList = []
    for text in textList:
        newText = deleteWord(text)
        newList.append(newText)
        targetFile.write(newText + "\n")

    #Close the file
    targetFile.close()
    
    return newList

Se va a realizar este proceso tres veces para generar más textos distintos de los que se posteriormente se elegirán los mejores.

In [12]:
#Import the original data
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

#Apply the synonym replacement 3 times to both datasets
for i in range(3):
    validPath = f"6. Random Deletion/1. All Augmented Data/validRandomDeletionReviews{i + 1}.txt"
    invalidPath = f"6. Random Deletion/1. All Augmented Data/invalidRandomDeletionReviews{i + 1}.txt"

    
    randomDeletion(validOriginal, validPath)
    randomDeletion(invalidOriginal, invalidPath)

## Análisis de la eliminación aleatoria y selección de los datos

Importar los datos originales

In [11]:
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

Importar los datasets generados en la eliminacion aleatoria

In [13]:
#import all the data 
validRandomDeletionList = []
invalidRandomDeletionList = []
for i in range(3):
    validPath = f"6. Random Deletion/1. All Augmented Data/validRandomDeletionReviews{i + 1}.txt"
    invalidPath = f"6. Random Deletion/1. All Augmented Data/invalidRandomDeletionReviews{i + 1}.txt"

    validRandomDeletionList.append(importFromTxtToList(validPath))
    invalidRandomDeletionList.append(importFromTxtToList(invalidPath))

Realizar el análisis y la selección de los datos

In [14]:
validWithOriginalPath = '6. Random Deletion/2. Augmented Data/ ValidRandomDeletionWithOriginal.csv'
validAugmentedPath = '6. Random Deletion/2. Augmented Data/ValidRandomDeletionData.txt'
invalidWithOriginalPath = '6. Random Deletion/2. Augmented Data/InvalidRandomDeletionWithOriginal.csv'
invalidAugmentedPath = '6. Random Deletion/2. Augmented Data/InvalidRandomDeletionData.txt'

infoValid = processAugmentation(validOriginal, validRandomDeletionList, validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, invalidRandomDeletionList, invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [16]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...","""Tiene fácil acceso para las personas con movi...",0.976980,0.866667,"""Tiene fácil acceso para las personas con movi...",0.945306,0.866667,"""Tiene fácil acceso para las personas con movi...",0.976980,0.866667,"""Tiene fácil acceso para las con movilidad una...",0.951568,0.866667
1,"""Espero que hayan mejorais""","""que hayan mejorais""",0.915591,0.666667,"""que hayan mejorais""",0.915591,0.666667,"""Espero que mejorais""",0.900617,0.666667,"""que hayan mejorais""",0.915591,0.666667
2,"""La estación es antigua, aparte de tener una s...","""La estación es antigua, aparte de tener una s...",0.989588,0.863636,"""La estación es antigua, aparte de tener una s...",0.989588,0.863636,"""La estación es antigua, aparte de una sola sa...",0.987141,0.863636,"""La estación es antigua, aparte de una sola sa...",0.944223,0.909091
3,"""Bien""","""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000,"""Bien""",1.000000,1.000000
4,"""Bonito comodo""","""comodo""",0.800290,0.500000,"""Bonito""",0.791561,0.500000,"""comodo""",0.800290,0.500000,"""Bonito""",0.791561,0.500000


In [17]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...","""He 35 años en el barrio y que el metro nos di...",0.955221,0.818182,"""He vivido 35 años en el y reconozco que el me...",0.966433,0.863636,"""He vivido 35 años en el barrio y reconozco qu...",0.948992,0.863636,"""He 35 años en el barrio y que el metro nos di...",0.955221,0.818182
1,"""localización con muchos bares interesantes""","""localización con muchos interesantes""",0.846651,0.666667,"""localización con muchos interesantes""",0.846651,0.666667,"""con muchos bares interesantes""",0.734324,0.666667,"""con muchos bares interesantes""",0.734324,0.666667
2,"""…""","""…""",1.000000,0.000000,"""…""",1.000000,0.000000,"""…""",1.000000,0.000000,"""…""",1.000000,0.000000
3,"""Muy rica comida..""","""Muy comida..""",0.739944,0.666667,"""Muy comida..""",0.739944,0.666667,"""Muy comida..""",0.739944,0.666667,"""Muy comida..""",0.739944,0.666667
4,"""Estación del.metro""","""del.metro""",0.768227,0.500000,"""Estación""",0.710799,0.500000,"""Estación""",0.710799,0.500000,"""del.metro""",0.768227,0.500000


Éste método es el que más variación genera de los anteriores tres.

## Combinación de reemplazo por sinónimos, inserción, intercambio y eliminacion 
Para intentar que haya la mayor variación posibles, manteniendo la semántica de los textos, se van a combinar los cuatro anteriores métodos para evitar un posible overfiting cuando se entrene al modelo con estos datos.

In [25]:
#performs all EDA transformations
def mixedEDAMethods(textList, prob, targetFolder, fileName, numVersion):
    targetPath = targetFolder + "/" + fileName + str(numVersion) + ".txt"
    
    newList = synonymReplacement(textList, prob, targetFolder + "/1. Intermidiate Augmentation/" + fileName + str(numVersion) + "Synonym.txt")
    newList = randomInsertion(newList, targetFolder + "/1. Intermidiate Augmentation/" + fileName + str(numVersion) + "Insertion.txt")
    newList = randomSwap(newList, targetFolder + "/1. Intermidiate Augmentation/" + fileName + str(numVersion) + "Swap.txt")
    newList = randomDeletion(newList, targetPath)
    
    return newList

Se aplica éste método tres veces para generar más datos y despúes escoger los maś convenientes.

In [29]:
#Import the original data
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

#Apply the synonym replacement 3 times to both datasets
targetFolder = "7. Mixed EDA/1. All Augmented Data"
for i in range(3):
    prob = 0.25 + 2 * i / 10
    
    mixedEDAMethods(validOriginal, prob, targetFolder, "validMixedEDAReviews", i + 1)
    mixedEDAMethods(invalidOriginal, prob, targetFolder, "invalidMixedEDAReviews", i + 1)

## Análisis de la combinación de los métodos y selección de los datos

Importar los datos originales

In [30]:
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

Importar los datasets generados en la eliminacion aleatoria

In [31]:
#import all the data 
validMixedEDAList = []
invalidMixedEDAList = []
for i in range(3):
    validPath = f"7. Mixed EDA/1. All Augmented Data/validMixedEDAReviews{i + 1}.txt"
    invalidPath = f"7. Mixed EDA/1. All Augmented Data/invalidMixedEDAReviews{i + 1}.txt"

    validMixedEDAList.append(importFromTxtToList(validPath))
    invalidMixedEDAList.append(importFromTxtToList(invalidPath))

Realizar el análisis y la selección de los datos

In [32]:
validWithOriginalPath = '7. Mixed EDA/2. Augmented Data/ ValidMixedEDAWithOriginal.csv'
validAugmentedPath = '7. Mixed EDA/2. Augmented Data/ValidMixedEDAData.txt'
invalidWithOriginalPath = '7. Mixed EDA/2. Augmented Data/InvalidMixedEDAWithOriginal.csv'
invalidAugmentedPath = '7. Mixed EDA/2. Augmented Data/InvalidMixedEDAData.txt'

infoValid = processAugmentation(validOriginal, validMixedEDAList, validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, invalidMixedEDAList, invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [33]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...","""Tiene fácil movilidad para las salida pueblo ...",0.909179,0.705882,"""tiene fácil acceso para las personas con movi...",0.958459,0.875000,"""Tiene fácil movilidad para las salida pueblo ...",0.909179,0.705882,"""Tiene del entrada para fácil las personas con...",0.849175,0.647059
1,"""Espero que hayan mejorais""","""espero ""Espero que mejorais""""",0.858184,0.666667,"""mejorais que hayan Espero""",0.991634,1.000000,"""espero ""Espero que mejorais""""",0.858184,0.666667,"""que Espero hayan mejorais""",0.994041,1.000000
2,"""La estación es antigua, aparte de tener una s...","""La (que antigua, a un lado de dar a subterran...",0.788468,0.483871,"""""La estación sola pertenece aparte de ademas ...",0.858311,0.954545,"""""La estación metro"" pueblo aparte de tener un...",0.878832,0.760000,"""La (que antigua, a un lado de dar a subterran...",0.788468,0.483871
3,"""Bien""","""bien""",1.000000,1.000000,"""bien""",1.000000,1.000000,"""bien""",1.000000,1.000000,"""""Bien""""",0.963228,1.000000
4,"""Bonito comodo""","""comodo comodo""""",0.803378,0.500000,"""""Bonito comodo""""",0.986783,1.000000,"""""Bonito comodo""",0.991887,1.000000,"""comodo comodo""""",0.803378,0.500000


In [34]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...","""barrio desaparecido. 35 mucho tiempo en el pr...",0.897766,0.692308,"""He vivido 35 años en el barrio desaparecido y...",0.919991,0.720000,"""tren 35 tiempo zonas en el mejoró y reconozco...",0.851737,0.720000,"""barrio desaparecido. 35 mucho tiempo en el pr...",0.897766,0.692308
1,"""localización con muchos bares interesantes""","""localizacion bares con muchos ""localización""",0.917731,0.666667,"""interesantes con muchos localización interesa...",0.798462,0.666667,"""bares con muchos localización interesantes""",0.985325,1.000000,"""localizacion bares con muchos ""localización""",0.917731,0.666667
2,"""…""","""…""",1.000000,0.000000,"""…""",1.000000,0.000000,"""…""",1.000000,0.000000,"""…""",1.000000,0.000000
3,"""Muy rica comida..""","""rica rica Muy""",0.872533,0.666667,"""Muy comida.. rica""",0.980956,1.000000,"""rica rica Muy""",0.872533,0.666667,"""""Muy comida.."" rica""",0.942068,1.000000
4,"""Estación del.metro""","""Estación del.metro""",1.000000,1.000000,"""Estación del.metro""",1.000000,1.000000,"""del.metro Estación""",0.988279,1.000000,"""Estación del.metro""",1.000000,1.000000


Como se puede ver, los resultados generados son mucho más prometedores ya que genrean más variabilidad léxica pero mantienen bastante alta la similitud semántica.

## Albumentation
Consiste en cambiar el orden de las frases de un texto y eliminar las repetidas

Función que obtiene las frases de un texto

In [14]:
def getUniqueSentences(text):
    # Remove quotes from the input line to ensure clean processing
    newText = removeQuotes(text)
    # Split the cleaned line into sentences using '.' as the delimiter and create a set comprehension to ensure unique sentences
    sentencesSet = {sentence.strip() + "." for sentence in newText.split('.') if sentence.strip()}
    # Return the set of unique sentences
    
    return sentencesSet

Función que cambia el orden de las frases

In [15]:
def mixSentences(sentencesSet):
    # Convert the input set of sentences into a list for shuffling
    newList = list(sentencesSet)
    # Shuffle the list in place to randomize the order of sentences
    random.shuffle(newList)
    # Join the shuffled sentences into a single string, adding quotes around it
    return addQuotes(' '.join(newList))

Función que realiza la "albumentation" a todos lo elementos de una lista de textos

In [16]:
#NLP albumentation method
def albumentation(textList, targetPath):
    #Open the target file
    targetFile = open(targetPath, "w", encoding = 'utf-8')
    
    newList = []
    for text in textList:
        newText = mixSentences(getUniqueSentences(text))
        newList.append(newText)
        targetFile.write(newText + "\n")

    #Close the file
    targetFile.close()
    
    return newList

Se va a realizar este proceso tres veces para tener más datos y luego poder escoger los mejores

In [17]:
#Import the original data
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

#Apply the synonym replacement 3 times to both datasets
for i in range(3):
    validPath = f"8. Albumentation/1. All Augmented Data/validAlbumentationReviews{i + 1}.txt"
    invalidPath = f"8. Albumentation/1. All Augmented Data/invalidAlbumentationReviews{i + 1}.txt"

    
    albumentation(validOriginal, validPath)
    albumentation(invalidOriginal, invalidPath)

## Análisis del "albumentation" y selección de los datos

Importar los datos originales.

In [18]:
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validOriginal = importFromTxtToList(validOriginalPath)
invalidOriginal = importFromTxtToList(invalidOriginalPath) 

Importar los datasets generados en el intercambio aleatorio.

In [19]:
#import all the data 
validAlbumentationList = []
invalidAlbumentationList = []
for i in range(3):
    validPath = f"8. Albumentation/1. All Augmented Data/validAlbumentationReviews{i + 1}.txt"
    invalidPath = f"8. Albumentation/1. All Augmented Data/invalidAlbumentationReviews{i + 1}.txt"

    validAlbumentationList.append(importFromTxtToList(validPath))
    invalidAlbumentationList.append(importFromTxtToList(invalidPath))

Realizar el análisis y la selección de los datos

In [20]:
validWithOriginalPath = '8. Albumentation/2. Augmented Data/ ValidAlbumentationWithOriginal.csv'
validAugmentedPath = '8. Albumentation/2. Augmented Data/ValidAlbumentationData.txt'
invalidWithOriginalPath = '8. Albumentation/2. Augmented Data/InvalidAlbumentationWithOriginal.csv'
invalidAugmentedPath = '8. Albumentation/2. Augmented Data/InvalidAlbumentationData.txt'

infoValid = processAugmentation(validOriginal, validAlbumentationList, validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidOriginal, invalidAlbumentationList, invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [21]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""Tiene fácil acceso para las personas con movi...","""Tiene fácil acceso para las personas con movi...",1.000000,1.0,"""Tiene fácil acceso para las personas con movi...",1.000000,1.0,"""Tiene fácil acceso para las personas con movi...",1.000000,1.0,"""Tiene fácil acceso para las personas con movi...",1.000000,1.0
1,"""Espero que hayan mejorais""","""Espero que hayan mejorais.""",0.978180,1.0,"""Espero que hayan mejorais.""",0.978180,1.0,"""Espero que hayan mejorais.""",0.978180,1.0,"""Espero que hayan mejorais.""",0.978180,1.0
2,"""La estación es antigua, aparte de tener una s...","""La estación es antigua, aparte de tener una s...",0.998383,1.0,"""Además comunica por un pasillo subterráneo (q...",0.941534,1.0,"""La estación es antigua, aparte de tener una s...",0.998383,1.0,"""Además comunica por un pasillo subterráneo (q...",0.941534,1.0
3,"""Bien""","""Bien.""",0.909274,1.0,"""Bien.""",0.909274,1.0,"""Bien.""",0.909274,1.0,"""Bien.""",0.909274,1.0
4,"""Bonito comodo""","""Bonito comodo.""",0.956156,1.0,"""Bonito comodo.""",0.956156,1.0,"""Bonito comodo.""",0.956156,1.0,"""Bonito comodo.""",0.956156,1.0


In [22]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1,augmented2,semanticSimilarity2,lexicalSimilarity2,augmented3,semanticSimilarity3,lexicalSimilarity3
0,"""He vivido 35 años en el barrio y reconozco qu...","""He vivido 35 años en el barrio y reconozco qu...",0.995105,1.00,"""Una pena. La gente joven se ha ido a otras zo...",0.966767,1.00,"""He vivido 35 años en el barrio y reconozco qu...",0.995105,1.00,"""Desgraciadamente el barrio que conocí práctic...",0.959858,1.00
1,"""localización con muchos bares interesantes""","""localización con muchos bares interesantes.""",0.972995,1.00,"""localización con muchos bares interesantes.""",0.972995,1.00,"""localización con muchos bares interesantes.""",0.972995,1.00,"""localización con muchos bares interesantes.""",0.972995,1.00
2,"""…""","""….""",0.911401,0.00,"""….""",0.911401,0.00,"""….""",0.911401,0.00,"""….""",0.911401,0.00
3,"""Muy rica comida..""","""Muy rica comida.""",0.995217,1.00,"""Muy rica comida.""",0.995217,1.00,"""Muy rica comida.""",0.995217,1.00,"""Muy rica comida.""",0.995217,1.00
4,"""Estación del.metro""","""Estación del. metro.""",0.987054,0.25,"""Estación del. metro.""",0.987054,0.25,"""metro. Estación del.""",0.973336,0.25,"""metro. Estación del.""",0.973336,0.25


En este caso, la similitud semántica varía ligeramente, pero la similitud léxica permanece intacta, ya que no se introducen nuevas palabras ni se eliminan palabras existentes.

# Aumento de datos con modelos preentrenados
Para realizar esta sección del aumento de datos se va a seguir el artículo "Dara Augmentation Using Pre-trained Transformer Models".

## Imports necesarios

In [2]:
from datasets import DatasetDict, Dataset, load_dataset, concatenate_datasets
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq 

## GPT-2

### Funciones que se van a utilizar

Separación del conjunto de datos en: train, test y validation

In [3]:
def trainTestValidationSplit(dataset, trainPctg = 0.8):
    datasetSplit = dataset.train_test_split(test_size = 1 - trainPctg, seed = 54)
    validationTestSplit = datasetSplit["test"].train_test_split(test_size = 0.5, seed = 54)
    
    return DatasetDict({
        "train" : datasetSplit["train"],
        "test" : validationTestSplit["test"],
        "validation" : validationTestSplit["train"]
        })

Importar datos de un fichero (donde los datos no tienen etiquetas) a una lista de diccionarios de la forma: "texto" : texto

In [4]:
def importFromTxtToDictList(source):
    dictList = []
    with open(source, 'r', encoding="utf-8") as file:
        for line in file: 
            dictList.append({"text" : line.strip()})
            
    return dictList

Importar datos con etiquetas, donde las etiquetas están dadas por la carpeta de origen. Se genera una lista de diccionarios de la forma: "texto" : label + texto

In [5]:
def importFromTxtToDictListWithLabel(source, label):
    dictList = []
    
    #Open and store evety line in the file in a list
    with open(source, "r", encoding = 'utf-8') as dataFile:
        for line in dataFile:
            dictList.append({"text" : label + " : " + line.strip()})

    return dictList

Importar datos de dos ficheros (donde los datos no tienen etiquetas) a una lista de diccionarios de la forma: "original" : texto , "parafrasis" : texto

In [6]:
def importParaphraseFromTxtToDictList(originalPath, paraphrasePath):
    dictList = []
    
    #As both files have the same number of lines, they can be iterated simultaneously
    with open(originalPath, 'r', encoding = 'utf-8') as originalFile, open(paraphrasePath, 'r', encoding = 'utf-8') as paraphraseFile:
        for originalLine, paraphraseLine in zip(originalFile, paraphraseFile): 
            dictList.append({"original" : originalLine.strip(), "paraphrase" : paraphraseLine.strip()})
            
    return dictList

Importar datos de dos ficheros (donde los datos no tienen etiquetas) a una lista de diccionarios de la forma: "original" : label + texto , "parafrasis" : label + texto

In [7]:
def importParaphraseFromTxtToDictListWithLabel(originalPath, paraphrasePath, label):
    dictList = []
    
    #As both files have the same number of lines, they can be iterated simultaneously
    with open(originalPath, 'r', encoding = 'utf-8') as originalFile, open(paraphrasePath, 'r', encoding = 'utf-8') as paraphraseFile:
        for originalLine, paraphraseLine in zip(originalFile, paraphraseFile): 
            dictList.append({"original" : label + originalLine.strip(), "paraphrase" : label + paraphraseLine.strip()})
            
    return dictList

Tokenización de los datos donde solo se tiene en cuenta la clave text

In [8]:
def tokenizeFunction(examples, tokenizer):
    return tokenizer(examples["text"], truncation = True, padding = "max_length", max_length = 512)

Tokenización de datos para paráfrasis

In [9]:
def tokenizeFunctionParaphrases(examples, tokenizer):
    return tokenizer(
        f"<start> Original: {examples["original"]} <sep> Paráfrasis: {examples["paraphrase"]} <end>", 
        truncation = True, padding = "max_length", max_length = 512)
        

Entrenamiento de un modelo GPT2

In [10]:
def trainModel(savingPath, modelName, version, tokenizeFunction, dataDict, aditionalTokens = None, trainEpochs = 3, lr = 5e-5, freezePctg = 0):
    #Download the pretrained model and the tokenizer
    model = AutoModelForCausalLM.from_pretrained(modelName)
    tokenizer = AutoTokenizer.from_pretrained(modelName)
    
    #Check if the tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
        model.resize_token_embeddings(len(tokenizer))

    #if necessary add more tokens
    if aditionalTokens != None:
        tokenizer.add_special_tokens(aditionalTokens)
        model.resize_token_embeddings(len(tokenizer))
        
    tokenizedDataDict = dataDict.map(lambda x: tokenizeFunction(x, tokenizer), batched = False, load_from_cache_file = False)

    #If necessary freeze the layers that do not have to be trained
    if freezePctg > 0:
        for name, param in model.named_parameters():
            if 'transformer.h.' in name:  # GPT2 has blocks called 'transformer.h.X'
                layer_num = int(name.split('.')[2])
                if layer_num < model.config.n_layer * freezePctg:  #Freeze 80 % of the layers
                    param.requires_grad = False

    # Define training arguments
    training_args = TrainingArguments(
        output_dir = savingPath + "/" + modelName.replace("/", "%"),
        overwrite_output_dir = True,
        num_train_epochs = trainEpochs,
        per_device_train_batch_size = 8,
        logging_steps = 10,
        eval_strategy = "epoch",
        save_strategy = "no",
        fp16 = True,
        gradient_accumulation_steps = 4,
        weight_decay=0.01,                
        warmup_steps=500,  
        learning_rate = lr,
        logging_dir = savingPath + "/" + modelName.replace("/", "%"),
    )

    dataCollator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = False,  # Is not a mask lenguage model
    )

    #Generates the labels automatically so that the model can predict the next token
    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = tokenizedDataDict["train"],
        eval_dataset = tokenizedDataDict["validation"],
        data_collator = dataCollator,
    )

    #Train the model
    torch.cuda.empty_cache()
    trainer.train()

    #Save the model
    tokenizer.save_pretrained(savingPath + "/" + modelName.replace("/", "%") + "v" + str(version))
    model.save_pretrained(savingPath + "/" + modelName.replace("/", "%") + "v" + str(version))

Generación de texto dado un modelo GPT2

In [11]:
def generateText(modelPath, text):
    #Load the model
    model = AutoModelForCausalLM.from_pretrained(modelPath)
    tokenizer = AutoTokenizer.from_pretrained(modelPath)

    #Check if the tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
        model.resize_token_embeddings(len(tokenizer))

    #Tokenize the input text
    inputIds = tokenizer.encode(text, return_tensors = "pt")
    inputLength = inputIds.shape[-1]

    #Generate the text
    output = model.generate(inputIds, max_new_tokens = inputLength, num_return_sequences = 1, 
                            pad_token_id = tokenizer.pad_token_id, eos_token_id = tokenizer.eos_token_id, 
                            temperature = 0.7, top_k = 50, top_p = 0.9, repetition_penalty = 2.0,
                            no_repeat_ngram_size = 3,)

    #Decode the text
    generatedText = tokenizer.decode(output[0], skip_special_tokens=True)

    return generatedText

### Primer enfoque
Se van a hacer el refinamiento del modelo preentrenado GPT-2. El modelo es una versión ligera de GPT2 devido a las limitaciones de HW.

El propósito es adaptar el modelo a nuestro conjunto de datos y que sea capaz de generar una frase nueva semánticamente similar a la frase de entrada pero léxicamente distinta.

#### Preparación de los datos

Se importan los datos sin etiquetas

In [12]:
path = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/AllUnlabeledReviews.txt"

#Each element is a dictionary with the key text and the text as the value
dataDictList = importFromTxtToDictList(path)
#The list of dictionaries is converted to a dataset
dataset = Dataset.from_list(dataDictList)

#Split the data
dataDict = trainTestValidationSplit(dataset)

In [13]:
print(dataDict)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2587
    })
    test: Dataset({
        features: ['text'],
        num_rows: 324
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 323
    })
})


Se va a generar un segundo conjunto de datos siguiendo el artículo: Data Augmentation Using Pre-trained Transformer Models. Se van a anteponer las etiquetas de las clases a los ejemplos de las clases. Es decir, los datos que se van a proporcionar al modelo tendrán la siguiente forma: etiqueta : texto.

Originalmente, las etiquetas del cojunto de datos eran v (valida) y n (no válida). Sin embargo, para proporcionar más contexto semántico al modelo, se van a extender a valida y noValida.

In [14]:
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"

validReviewsLabeledDictList = importFromTxtToDictListWithLabel(validOriginalPath, "valida")
invalidReviewsLabeledDictList = importFromTxtToDictListWithLabel(invalidOriginalPath, "noValida")

In [15]:
#Each element is a dictionary with the key text and the text as the value
dataLabeledDictList = validReviewsLabeledDictList + invalidReviewsLabeledDictList
random.shuffle(dataLabeledDictList)

#The list of dictionaries is converted to a dataset
labeledDataset = Dataset.from_list(dataLabeledDictList)

#Split the data
labeledDataDict = trainTestValidationSplit(labeledDataset)

In [16]:
print(labeledDataDict)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2587
    })
    test: Dataset({
        features: ['text'],
        num_rows: 324
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 323
    })
})


#### Entrenamiento de los modelos
Se van a entrenar varios modelos con distintas características para luego poder compararlos entre sí

In [17]:
savingPath = "/home/ibon/Documentos/1. Models/GPT2/1. Generation"
modelName = "mrm8488/spanish-gpt2"

#Possible characteristics of the models
dataDictList = [dataDict, labeledDataDict]
freezePctgList = [0.8, 0.5, 0]
trainEpochsList = [3, 6]

#Version control
version = 0

for i, dataDict in enumerate(dataDictList):
    for freezePctg in freezePctgList:
        for trainEpochs in trainEpochsList:
            trainModel(savingPath, modelName, version, tokenizeFunction, dataDict, trainEpochs = trainEpochs, freezePctg = freezePctg)

            with open(savingPath + "/VersionControlGPT2Generation.csv", "a") as file:
                data = "unlabeled \n" if i == 0 else "labeled \n"

                #Update the version control
                file.write(modelName.replace("/", "%") + "v" + str(version) + "," + str(version) + "," + str(freezePctg) + "," + str(trainEpochs) + "," + data)

            version += 1

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Map: 100%|██████████| 323/323 [00:00<00:00, 9594.35 examples/s]


Epoch,Training Loss,Validation Loss
1,18.704100,4.752882
2,17.701900,4.576190
3,17.280200,4.401620


Map: 100%|██████████| 323/323 [00:00<00:00, 8722.32 examples/s]


Epoch,Training Loss,Validation Loss
1,18.703700,4.753025
2,17.702000,4.576076
3,17.279400,4.401365
4,16.816200,4.259889
5,15.998100,4.146826
6,15.587200,4.070008


Map: 100%|██████████| 323/323 [00:00<00:00, 8908.85 examples/s]


...

Epoch,Training Loss,Validation Loss
1,17.319600,4.353586
2,14.889600,3.646158
3,13.825400,3.469061
4,13.259400,3.405213
5,12.564400,3.367789
6,12.000400,3.349270


Map: 100%|██████████| 323/323 [00:00<00:00, 7921.46 examples/s]


Epoch,Training Loss,Validation Loss
1,15.405300,3.800304
2,14.120800,3.473247
3,13.443000,3.396134


Map: 100%|██████████| 323/323 [00:00<00:00, 8153.20 examples/s]


Epoch,Training Loss,Validation Loss
1,15.405000,3.800235
2,14.120900,3.473225
3,13.443600,3.396276
4,12.833800,3.348695
5,12.098500,3.325533
6,11.417000,3.328719


#### Probar los modelos

In [18]:
savingPath = "/home/ibon/Documentos/1. Models/GPT2/1. Generation"
text = "Parafrasea: suele haber demasiada gente en el metro"

#For every generated model
with open(savingPath + "/VersionControlGPT2Generation.csv", "r") as file:
    #Ignore the first line of the file
    next(file)

    for line in file:
        elemsList = line.split(",")
        
        modelName = elemsList[0]
        modelPath = savingPath + "/" + modelName

        #Generate the text
        generatedText = generateText(modelPath, text)

        print("Modelo: " +  modelName + "\n")
        print("Texto: " + generatedText + "\n")

/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Modelo: mrm8488%spanish-gpt2v0

Texto: Parafrasea: suele haber demasiada gente en el metro, pero no hay nadie que se ocupe de los trenes

Modelo: mrm8488%spanish-gpt2v1

Texto: Parafrasea: suele haber demasiada gente en el metro, pero hay que tener cuidado de no perder la calma

Modelo: mrm8488%spanish-gpt2v2

Texto: Parafrasea: suele haber demasiada gente en el metro, pero no hay que olvidar la estación de metro.

Modelo: mrm8488%spanish-gpt2v3

Texto: Parafrasea: suele haber demasiada gente en el metro, por lo que es una estación muy transitada.

Modelo: mrm8488%spanish-gpt2v4

Texto: Parafrasea: suele haber demasiada gente en el metro, y no hay suficientes autobuses para todos los pasajeros.

Modelo: mrm8488%spanish-gpt2v5

Texto: Parafrasea: suele haber demasiada gente en el metro, por lo que hay que ir con cuidado."

Modelo: mrm8488%spanish-gpt2v6

Texto: Parafrasea: suele haber demasiada gente en el metro, pero no hay nadie que se quede con la boca

Modelo: mrm8488%spanish-gpt2v7

Ninguno de los modelos es capaz de parafrasear, todos completan la frase de entrada. Esto se debe a que no se ha entrenado el modelo para tal tarea. 

### Segundo Enfoque
En este caso, se van a entrenar los modelos de forma explícita para el parafraseo. Es decir, los datos de entrenamiento se van a organizar en pares de frases: original - parafraseo.

Dado que hemos aplicado técnicas de aumento de datos, disponemos de bastantes pares de frases. 

Se han aplicado 3 variaciones de la retrotraducción, por lo que solo con esto, se disponen de tres paraes de frases por cada elemento del conjunto.

#### Preparación de los datos

Se importan los datos de las 3 variaciones de la retrotraducción sin tener en cuenta las etiquetas

In [12]:
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"

backtranslationFolder = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/1. Back Translation/1. Google Translator"
invalidBacktranslationPath1 = backtranslationFolder + "/InvalidReviewsTranslationsEsEnEnEs.txt"
invalidBacktranslationPath2 = backtranslationFolder + "/InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
invalidBacktranslationPath3 = backtranslationFolder + "/InvalidReviewsTranslationsEsJaJaEs.txt"
validBacktranslationPath1 = backtranslationFolder + "/ValidReviewsTranslationsEsEnEnEs.txt"
validBacktranslationPath2 = backtranslationFolder + "/ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
validBacktranslationPath3 = backtranslationFolder + "/ValidReviewsTranslationsEsJaJaEs.txt"

invalidParaphrasesDictList1 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath1)
invalidParaphrasesDictList2 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath2)
invalidParaphrasesDictList3 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath3)
validParaphrasesDictList1 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath1)
validParaphrasesDictList2 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath2)
validParaphrasesDictList3 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath3)

#Join all the lists
paraphrasesDictList = invalidParaphrasesDictList1 + invalidParaphrasesDictList2 + invalidParaphrasesDictList3 + validParaphrasesDictList1 + validParaphrasesDictList2 + validParaphrasesDictList3
#Shuffle the lists
random.shuffle(paraphrasesDictList)

#Convert it to a Hugging Face dataset
paraphrasesDataset = Dataset.from_list(paraphrasesDictList)

#Split the data
paraphrasesDataDict = trainTestValidationSplit(paraphrasesDataset)

In [13]:
print(paraphrasesDataDict)

DatasetDict({
    train: Dataset({
        features: ['original', 'paraphrase'],
        num_rows: 7761
    })
    test: Dataset({
        features: ['original', 'paraphrase'],
        num_rows: 971
    })
    validation: Dataset({
        features: ['original', 'paraphrase'],
        num_rows: 970
    })
})


Se importan los datos de las 3 variaciones de la retrotraducción teniendo en cuenta las etiquetas. 

Originalmente, las etiquetas del cojunto de datos eran v (valida) y n (no válida). Sin embargo, para proporcionar más contexto semántico al modelo, se van a extender a valida y noValida.

In [14]:
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"

backtranslationFolder = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/1. Back Translation/1. Google Translator"
invalidBacktranslationPath1 = backtranslationFolder + "/InvalidReviewsTranslationsEsEnEnEs.txt"
invalidBacktranslationPath2 = backtranslationFolder + "/InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
invalidBacktranslationPath3 = backtranslationFolder + "/InvalidReviewsTranslationsEsJaJaEs.txt"
validBacktranslationPath1 = backtranslationFolder + "/ValidReviewsTranslationsEsEnEnEs.txt"
validBacktranslationPath2 = backtranslationFolder + "/ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
validBacktranslationPath3 = backtranslationFolder + "/ValidReviewsTranslationsEsJaJaEs.txt"

invalidLabeledParaphrasesDictList1 = importParaphraseFromTxtToDictListWithLabel(invalidOriginalPath, invalidBacktranslationPath1, "noValida")
invalidLabeledParaphrasesDictList2 = importParaphraseFromTxtToDictListWithLabel(invalidOriginalPath, invalidBacktranslationPath2, "noValida")
invalidLabeledParaphrasesDictList3 = importParaphraseFromTxtToDictListWithLabel(invalidOriginalPath, invalidBacktranslationPath3, "noValida")
validLabeledParaphrasesDictList1 = importParaphraseFromTxtToDictListWithLabel(validOriginalPath, validBacktranslationPath1, "valida")
validLabeledParaphrasesDictList2 = importParaphraseFromTxtToDictListWithLabel(validOriginalPath, validBacktranslationPath2, "valida")
validLabeledParaphrasesDictList3 = importParaphraseFromTxtToDictListWithLabel(validOriginalPath, validBacktranslationPath3, "valida")

#Join all the lists
labeledParaphrasesDictList = invalidLabeledParaphrasesDictList1 + invalidLabeledParaphrasesDictList2 + invalidLabeledParaphrasesDictList3 + validLabeledParaphrasesDictList1 + validLabeledParaphrasesDictList2 + validLabeledParaphrasesDictList3
#Shuffle the lists
random.shuffle(labeledParaphrasesDictList)

#Convert it to a Hugging Face dataset
labeledParaphrasesDataset = Dataset.from_list(labeledParaphrasesDictList)

#Split the data
labeledParaphrasesDataDict = trainTestValidationSplit(labeledParaphrasesDataset)

In [15]:
print(labeledParaphrasesDataDict)

DatasetDict({
    train: Dataset({
        features: ['original', 'paraphrase'],
        num_rows: 7761
    })
    test: Dataset({
        features: ['original', 'paraphrase'],
        num_rows: 971
    })
    validation: Dataset({
        features: ['original', 'paraphrase'],
        num_rows: 970
    })
})


#### Entrenamiento de los modelos
Se van a entrenar varios modelos con distintas características para luego poder compararlos entre sí

In [16]:
savingPath = "/home/ibon/Documentos/1. Models/GPT2/2. Paraphrasing"
modelName = "mrm8488/spanish-gpt2"

#Generate the aditional tokens for the separation of the original and paraphrased texts
aditionalTokens = {'additional_special_tokens': ['<start>', '<sep>', '<end>']}

#Possible characteristics of the models
dataDictList = [paraphrasesDataDict, labeledParaphrasesDataDict]
freezePctgList = [0.8, 0.5, 0]
trainEpochsList = [3, 6]

#Version control
version = 0

for i, dataDict in enumerate(dataDictList):
    for freezePctg in freezePctgList:
        for trainEpochs in trainEpochsList:
            trainModel(savingPath, modelName, version, tokenizeFunctionParaphrases, dataDict, aditionalTokens = aditionalTokens, trainEpochs = trainEpochs, freezePctg = freezePctg)

            with open(savingPath + "/VersionControlGPT2Paraphrasing.csv", "a") as file:
                data = "unlabeled \n" if i == 0 else "labeled \n"

                #Update the version control
                file.write(modelName.replace("/", "%") + "v" + str(version) + "," + str(version) + "," + str(freezePctg) + "," + str(trainEpochs) + "," + data)

            version += 1

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Map: 100%|██████████| 970/970 [00:00<00:00, 3918.43 examples/s]


Epoch,Training Loss,Validation Loss
0,12.078500,3.017042
1,10.306700,2.605485
2,9.327900,2.290071


Map: 100%|██████████| 970/970 [00:00<00:00, 3981.81 examples/s]


Epoch,Training Loss,Validation Loss
0,12.078500,3.016943
1,10.307600,2.605726
2,9.101600,2.228823
3,8.621400,2.174544
4,8.181700,2.151142
5,7.977200,2.143585


Map: 100%|██████████| 970/970 [00:00<00:00, 3883.51 examples/s]


...

Epoch,Training Loss,Validation Loss
0,10.393600,2.564410
1,9.321600,2.276447
2,7.846500,1.921246
3,7.093000,1.861821
4,6.862300,1.834509
5,6.919800,1.826032


Map: 100%|██████████| 970/970 [00:00<00:00, 3863.29 examples/s]


Epoch,Training Loss,Validation Loss
0,9.867100,2.440304
1,8.709900,2.118285
2,7.448900,1.846492


Map: 100%|██████████| 970/970 [00:00<00:00, 3935.85 examples/s]


Epoch,Training Loss,Validation Loss
0,9.865300,2.439883
1,8.708000,2.117530
2,7.292000,1.804644
3,6.356300,1.734845
4,5.996300,1.700825
5,5.971100,1.691810


#### Probar los modelos

In [17]:
savingPath = "/home/ibon/Documentos/1. Models/GPT2/2. Paraphrasing"
text = "<start> Original: El metro es rapido <sep> Paráfrasis: "

#For every generated model
with open(savingPath + "/VersionControlGPT2Paraphrasing.csv", "r") as file:
    #Ignore the first line of the file
    next(file)

    for line in file:
        elemsList = line.split(",")
        
        modelName = elemsList[0]
        modelPath = savingPath + "/" + modelName

        #Generate the text
        generatedText = generateText(modelPath, text)

        print("Modelo: " +  modelName + "\n")
        print("Texto: " + generatedText + "\n")

/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Modelo: mrm8488%spanish-gpt2v0

Texto:  Original: El metro es rapido  Paráfrasis:  Parámonos en el Metro de Madrid, cerca del aeropuerto. En

Modelo: mrm8488%spanish-gpt2v0

Texto:  Original: El metro es rapido  Paráfrasis:  Parámonos en el Metro de Madrid, cerca del aeropuerto. En

Modelo: mrm8488%spanish-gpt2v1

Texto:  Original: El metro es rapido  Paráfrasis:  Parámonos en la estación de Metro. No hay trenes directos desde

Modelo: mrm8488%spanish-gpt2v2

Texto:  Original: El metro es rapido  Paráfrasis:  Parágrafo: La línea de metro está en construcción. Es una

Modelo: mrm8488%spanish-gpt2v3

Texto:  Original: El metro es rapido  Paráfrasis:  Parágrafo: La línea de metro no tiene cobertura. Es una

Modelo: mrm8488%spanish-gpt2v4

Texto:  Original: El metro es rapido  Paráfrasis:  que el metro no tiene paradas en la estación de tren. Es una pena

Modelo: mrm8488%spanish-gpt2v5

Texto:  Original: El metro es rapido  Paráfrasis:  que el metro no tiene paradas en las vías. Es una est

Como se puede ver, los resultados siguen siendo muy malos. El modelo no genera la paráfrasis y muchas veces genera texto incoherente. Esto puede deberse a varias razones: el formato y la calidad de los datos de entrenamientos no son lo suficinetemente buenos o cuantiosos, los hiperparámetros de generación no son óptimos o la opción más probable: el modelo base es inadecuado (GPT2 no fue diseñado para paráfrasis).

## T5

Dado que GPT2 no está optimizado para parafraser, sino que se usa para generar texto, se va a usar un modelo text2text llamado t5 y desarrollado por Google.

Como las versiones de t5 normales proporcionadas por Google no estan pensadas para usarse en castellano, se va a usar una versión mejorada y con más idiomas (incluido el castellano), llamado flan-t5, que además está entrenado para realizar más tareas sin necesidad de hacer fine-tuning (aunque esto mejora el rendimiento).

### Funciones que se van a utilizar

Separación del conjunto de datos en: train, test y validation

In [3]:
def trainTestValidationSplit(dataset, trainPctg = 0.8):
    datasetSplit = dataset.train_test_split(test_size = 1 - trainPctg, seed = 54)
    validationTestSplit = datasetSplit["test"].train_test_split(test_size = 0.5, seed = 54)
    
    return DatasetDict({
        "train" : datasetSplit["train"],
        "test" : validationTestSplit["test"],
        "validation" : validationTestSplit["train"]
        })

Importar datos de dos ficheros (donde los datos no tienen etiquetas) a una lista de diccionarios de la forma: "original" : texto , "parafrasis" : texto

In [4]:
def importParaphraseFromTxtToDictList(originalPath, paraphrasePath):
    dictList = []
    
    #As both files have the same number of lines, they can be iterated simultaneously
    with open(originalPath, 'r', encoding = 'utf-8') as originalFile, open(paraphrasePath, 'r', encoding = 'utf-8') as paraphraseFile:
        for originalLine, paraphraseLine in zip(originalFile, paraphraseFile): 
            dictList.append({"sentence1" : originalLine.strip(), "sentence2" : paraphraseLine.strip()})
            
    return dictList

Importación y filtrado de PAWSX (dataset de Google), de forma que se obtienen las frases en castellano y únicamente los datos que son paráfrasis (etiqueta a 1). Además, se van a eliminar las columnas de id y label, ya que no serán necesarias.

In [5]:
def processPAWSX():
    #Import the dataset with the spanish phrases
    datasetDict = load_dataset('paws-x', 'es')

    #Create an empty dataset to save the paraphrases
    newDatasetDict = DatasetDict()

    #Filter the paraphrases
    for key, valueDataset in datasetDict.items():
        paraphraseDataset = valueDataset.filter(lambda example: example['label'] == 1)

        paraphraseDataset = paraphraseDataset.remove_columns(["id", "label"])
    
        newDatasetDict[key] = paraphraseDataset
    
    return newDatasetDict

Tokenización y preprocesamiento de los datos

In [6]:
def tokenizeFunctionFlanT5(examples, tokenizer):
    #Add the instruction to all the sentecnces for the model to understand what it has to do
    inputs = [f"Reescribe esta frase: {text}" for text in examples["sentence1"]]

    #Indicate that the targets are the paraphrased sentences
    targets = examples["sentence2"]

    #Tokenize the input
    model_inputs = tokenizer(inputs, max_length = 512, truncation = True, padding = "max_length")

    #Tokenize the output
    labels = tokenizer(targets, max_length = 512, truncation = True, padding = "max_length").input_ids

    #Add the expected output to the input of the model
    model_inputs["labels"] = labels
    
    return model_inputs

Entrenamiento para un modelo flan-t5

In [4]:
def trainFlanT5Model(savingPath, modelName, version, tokenizeFunction, dataDict, aditionalTokens = None, trainEpochs = 3, lr = 5e-5, freezePctg = 0):
    #Download the pretrained model and the tokenizer
    model = T5ForConditionalGeneration.from_pretrained(modelName)
    tokenizer = T5Tokenizer.from_pretrained(modelName)
    
    #Check if the tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
        model.resize_token_embeddings(len(tokenizer))

    #if necessary add more tokens
    if aditionalTokens != None:
        tokenizer.add_special_tokens(aditionalTokens)
        model.resize_token_embeddings(len(tokenizer))
        
    tokenizedDataDict = dataDict.map(lambda x: tokenizeFunction(x, tokenizer), batched = True, load_from_cache_file = False)

    #Remove the sentece1 and sentence2 columns
    newTokenizedDataDict = DatasetDict()
    
    for key in tokenizedDataDict.keys():
        newTokenizedData = tokenizedDataDict[key].remove_columns(["sentence1", "sentence2"])
    
        newTokenizedDataDict[key] = newTokenizedData

    tokenizedDataDict = newTokenizedDataDict

    #If necessary freeze the layers that do not have to be trained
    if freezePctg > 0:
        #Number of encoder and decoder layers
        numEncoderLayers = len(model.encoder.block)
        numDecoderLayers = len(model.decoder.block)

        #Compute how many layers have to be frozen
        numFreezeEncoder = int(freezePctg * numEncoderLayers)  
        numFreezeDencoder = int(freezePctg * numDecoderLayers)  
        
        # Congelar las capas del encoder
        for i, layer in enumerate(model.encoder.block):
            if i < numFreezeEncoder:
                for param in layer.parameters():
                    param.requires_grad = False
        
        # Congelar las capas del decoder
        for i, layer in enumerate(model.decoder.block):
            if i < numFreezeDencoder:
                for param in layer.parameters():
                    param.requires_grad = False


    # Define training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir = savingPath + "/" + modelName.replace("/", "%"),
        evaluation_strategy="epoch",
        learning_rate = lr,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs = trainEpochs,
        weight_decay=0.01,
        save_strategy = "no",
        predict_with_generate = True,
        logging_dir = savingPath + "/" + modelName.replace("/", "%"),
        logging_steps = 10,
        do_train=True,
        do_eval=True, 
        fp16 = False,
        overwrite_output_dir = True,
        gradient_accumulation_steps = 4,             
        warmup_steps = 500, 
    )

    dataCollator = DataCollatorForSeq2Seq(
        tokenizer = tokenizer,
        model = model,
        padding = True,
    )

    #Generates the labels automatically so that the model can predict the next token
    trainer = Seq2SeqTrainer(
        model = model,
        args = training_args,
        train_dataset = tokenizedDataDict["train"],
        eval_dataset = tokenizedDataDict["validation"],
        data_collator = dataCollator,
    )

    #Train the model
    torch.cuda.empty_cache()
    trainer.train()

    #Save the model
    tokenizer.save_pretrained(savingPath + "/" + modelName.replace("/", "%") + "v" + str(version))
    model.save_pretrained(savingPath + "/" + modelName.replace("/", "%") + "v" + str(version))

Generación de texto dado un modelo flan t5

In [10]:
def generateTextFlanT5(modelPath, text):
    #Load the model
    model = T5ForConditionalGeneration.from_pretrained(modelPath)
    tokenizer = T5Tokenizer.from_pretrained(modelPath)

    #Check if the tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
        model.resize_token_embeddings(len(tokenizer))

    #Tokenize the input text
    inputIds = tokenizer(text, return_tensors = "pt").input_ids
    inputLength = inputIds.shape[-1]

    #Generate the text
    outputs = model.generate(inputIds, num_return_sequences = 5, max_length = inputLength, do_sample = True, num_beams = 5, early_stopping = True, repetition_penalty = 1.5, no_repeat_ngram_size = 2, top_k = 50, top_p = 0.95, temperature = 1.5,)

    #Decode the text
    generatedText = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generatedText

### Preparación de los datos

##### PAWSX original

Para entrenar un modelo que tenga una precisión considerable, nos hemos dado cuenta de que los datos de los que disponemos son insuficientes. Consecuentemente, vamos a hacer uso del dataset PAWS-X de Google, donde es la versión extendida de PAWS en 7 idiomas.

In [9]:
#Import the dataset with the spanish phrases
datasetDict2 = load_dataset('paws-x', 'es')

In [10]:
print(datasetDict2)

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})


In [11]:
print(datasetDict2["train"][10])

{'id': 11, 'sentence1': 'Kabir Suman grabó varios álbumes con el nombre de Suman Chattopaddhyay o Suman Chatterjee entre 1992 y 1999.', 'sentence2': 'Suman Chatterjee, grabó varios álbumes entre 1992 y 1999 con el nombre de Suman Chattopaddhyay o Kabir Suman.', 'label': 0}


##### PAWSX filtrado
Dado que solo queremos hacer uso del modelo en castellano, únicamente nos quedaremos con las frases en este idioma. Además, el conjunto de datos original contiene pares de frases que son paráfrasis y que no lo son. En nuestro caso, únicamente nos interesan las primeras, por lo que también se filtrarán.

In [12]:
PAWSXDataDict = processPAWSX()

Estructura del dataset

In [13]:
print(PAWSXDataDict)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 21829
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 907
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 847
    })
})


Diez ejemplos no consecutivos para cerciorarnos de que todos los datos son paráfrasis

In [14]:
for i in range(10):
    print(PAWSXDataDict["train"][1050 + i * 2])

{'sentence1': 'El distrito electoral se encuentra en la bahía de Swansea, situada en la margen derecha del río Afan, cerca de su desembocadura en el sur de Gales.', 'sentence2': 'El distrito electoral se encuentra en Swansea Bay, en la orilla derecha del río Afan, cerca de su desembocadura en el sur de Gales.'}
{'sentence1': "Los `` Zapateros '' cayeron al 16º en 1991: 92, antes de caer en el 20º puesto en 1992: 93 bajo Phil Chard.", 'sentence2': "Los `` Zapateros '' cayeron al 16º en 1991: 92, antes de caer en el 20º puesto en 1992: 93 bajo Phil Chard."}
{'sentence1': 'La ley albanesa está codificada y basada en la ley francesa.', 'sentence2': 'La ley albanesa está codificada y se basa en la ley francesa.'}
{'sentence1': 'Nació en Usera, España (Madrid) el 18 de abril de 1976.', 'sentence2': 'Nació el 18 de abril de 1976 en Usera, España (Madrid).'}
{'sentence1': 'Los pequeños blenios marinos blenioides ("Ecsenius australianus") son peces australianos del género "Ecsenius".', 'sentenc

##### Dataset propio
Dado que hemos aplicado técnicas de aumento de datos, disponemos pares de frases (paráfrasis) adaptadas a nuestro dominio (reseñas de estaciones de metro). 

Se han aplicado 3 variaciones de la retrotraducción, por lo que solo con esto, se disponen de tres paraes de frases por cada elemento del conjunto.

In [15]:
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"

backtranslationFolder = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/1. Back Translation/1. Google Translator"
invalidBacktranslationPath1 = backtranslationFolder + "/InvalidReviewsTranslationsEsEnEnEs.txt"
invalidBacktranslationPath2 = backtranslationFolder + "/InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
invalidBacktranslationPath3 = backtranslationFolder + "/InvalidReviewsTranslationsEsJaJaEs.txt"
validBacktranslationPath1 = backtranslationFolder + "/ValidReviewsTranslationsEsEnEnEs.txt"
validBacktranslationPath2 = backtranslationFolder + "/ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
validBacktranslationPath3 = backtranslationFolder + "/ValidReviewsTranslationsEsJaJaEs.txt"

invalidParaphrasesDictList1 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath1)
invalidParaphrasesDictList2 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath2)
invalidParaphrasesDictList3 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath3)
validParaphrasesDictList1 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath1)
validParaphrasesDictList2 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath2)
validParaphrasesDictList3 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath3)

#Join all the lists
backtranslationDictList = invalidParaphrasesDictList1 + invalidParaphrasesDictList2 + invalidParaphrasesDictList3 + validParaphrasesDictList1 + validParaphrasesDictList2 + validParaphrasesDictList3
#Shuffle the lists
random.shuffle(backtranslationDictList)

#Convert it to a Hugging Face dataset
backtranslationDataset = Dataset.from_list(backtranslationDictList)

In [16]:
#Split the data
backtranslationDataDict = trainTestValidationSplit(backtranslationDataset)

##### Unión del PAWSX filtrado y el dataset propio

In [17]:
#Concatenate the datasets
unionDatasetDict = DatasetDict()

for key in PAWSXDataDict.keys():
    newDataset = concatenate_datasets([PAWSXDataDict[key], backtranslationDataDict[key]])

    unionDatasetDict[key] = newDataset

In [18]:
print(unionDatasetDict)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 29590
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 1878
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 1817
    })
})


### Entrenamiento de los modelos

In [19]:
savingPath = "/home/ibon/Documentos/1. Models/FlanT5Small"
modelName = "google/flan-t5-small"

#Possible characteristics of the models
dataDictList = [backtranslationDataDict, PAWSXDataDict, unionDatasetDict]
freezePctgList = [0.8, 0]
trainEpochsList = [6]

#Version control
version = 0

for i, dataDict in enumerate(dataDictList):
    for freezePctg in freezePctgList:
        for trainEpochs in trainEpochsList:
            trainFlanT5Model(savingPath, modelName, version, tokenizeFunctionFlanT5, dataDict, aditionalTokens = None, trainEpochs = trainEpochs, freezePctg = freezePctg)

            with open(savingPath + "/VersionControlFlanT5Small.csv", "a") as file:
                data = "unlabeled \n" if i == 0 else "labeled \n"

                #Update the version control
                file.write(modelName.replace("/", "%") + "v" + str(version) + "," + str(version) + "," + str(freezePctg) + "," + str(trainEpochs) + "," + data)

            version += 1

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|██████████| 970/970 [00:00<00:00, 4214.91 examples/s]
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(

Epoch,Training Loss,Validation Loss
0,78.753500,14.595844
1,2.974500,0.411336
2,0.704100,0.161291
3,0.558500,0.155704
4,0.754800,0.153961
5,0.734000,0.153505


Map: 100%|██████████| 970/970 [00:00<00:00, 4143.68 examples/s]
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,20.500100,4.170770
1,1.728500,0.234523
2,0.619900,0.149062
3,0.492800,0.143550
4,0.664000,0.141230
5,0.645800,0.140847


Map: 100%|██████████| 847/847 [00:00<00:00, 4408.93 examples/s]
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch,Training Loss,Validation Loss
0,0.314500,0.096967
1,0.260000,0.091664
2,0.240900,0.090016
4,0.244000,0.088631
5,0.235700,0.088624


Map: 100%|██████████| 1817/1817 [00:00<00:00, 4359.05 examples/s]
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.437000,0.134752
1,0.363300,0.130644
2,0.451000,0.127939
3,0.357400,0.126981
4,0.384200,0.126165
5,0.357800,0.126157


Map: 100%|██████████| 1817/1817 [00:00<00:00, 4098.90 examples/s]
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.387900,0.125870
1,0.321700,0.121482
2,0.396300,0.118064
3,0.309800,0.116897
4,0.327600,0.115868
5,0.308000,0.115681


### Probar los modelos

In [26]:
savingPath = "/home/ibon/Documentos/1. Models/FlanT5Small"
text = "Reescribe esta frase: La estación no tiene escaleras mecanicas, pero los metros vienen en hora. No es apta para invalidos"

#For every generated model
with open(savingPath + "/VersionControlFlanT5Small.csv", "r") as file:
    #Ignore the first line of the file
    next(file)

    for line in file:
        elemsList = line.split(",")
        
        modelName = elemsList[0]
        modelPath = savingPath + "/" + modelName

        #Generate the text
        generatedText = generateTextFlanT5(modelPath, text)

        print("Modelo: " +  modelName + "\n")
        print("Texto: " + generatedText + "\n")

Modelo: google%flan-t5-smallv0

Texto: “La estación no tiene escaleras mecanicos, pero las metros vienen enhora. No hay apta para invalidarios”.

Modelo: google%flan-t5-smallv1

Texto: La estación no tiene escaleras mecánicas, pero los metros vienen horas. No hay apta para invalidos

Modelo: google%flan-t5-smallv2

Texto: La estación no tiene escaleras mécaniques, pero los metros vienen hora. No se apta para invalidos.

Modelo: google%flan-t5-smallv3

Texto: La estación no tiene escaleras de forma mecanica, pero las metros vienen enhora. No és apta para invalidos.

Modelo: google%flan-t5-smallv4

Texto: La estación no tiene escaleras mécaniques, pero los metros vienen hora. No hay apta para invalidos.

Modelo: google%flan-t5-smallv5

Texto: La estación no tiene escaleras mecánicas, pero los metros vienen horas. No hay apta para invalidos



### Modificaciones

Como se puede ver, los modelos generan salidas idénticas o salidas poco coherentes. Consecuentemente, se va aplicar otro enfoque en el entrenamiento: se va a añadir de forma más explícita la tarea que debe realizar el modelo y se van a introducir prompts negativos para poder distinguir entre salidas correctas e incorrectas.

##### Añadir ejemplos negativos

In [5]:
def processPAWSXMod():
    #Import the dataset with the spanish phrases
    datasetDict = load_dataset('paws-x', 'es')

    #Create an empty dataset to save the paraphrases
    newDatasetDict = DatasetDict()

    #Filter the paraphrases
    for key, valueDataset in datasetDict.items():
        paraphraseDataset = valueDataset.filter(lambda example: example['label'] == 1)

        paraphraseDataset = paraphraseDataset.remove_columns(["id"])
        paraphraseDataset = paraphraseDataset.rename_column("label", "isPositive")
    
        newDatasetDict[key] = paraphraseDataset

    #Add the negatives
    for key, valueDataset in newDatasetDict.items():
        negativeExamples = []
        for value in valueDataset:
            if random.random() <= 0.4:
                negativeExamples.append({
                    "sentence1" : value["sentence1"],
                    "sentence2" : value["sentence1"],
                    "isPositive" : 0
                })
        newValueDataset = Dataset.from_list(negativeExamples)
        newValueDataset = newValueDataset.cast(valueDataset.features)
        newDatasetDict[key] = concatenate_datasets([valueDataset, newValueDataset])
    
    return newDatasetDict

In [6]:
PAWSXDataDictMod = processPAWSXMod()

Casting the dataset: 100%|██████████| 333/333 [00:00<00:00, 237252.12 examples/s]


In [7]:
print(PAWSXDataDictMod)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'isPositive'],
        num_rows: 30395
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'isPositive'],
        num_rows: 1294
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'isPositive'],
        num_rows: 1180
    })
})


##### Tokenización modificada

In [8]:
def tokenizeFunctionFlanT5Mod(examples, tokenizer):
    #Add the instruction to all the sentecnces for the model to understand what it has to do
    inputs = [f"Haz un parafraseo creativo de esta frase: {text}" for text in examples["sentence1"]]
    isPositive = examples["isPositive"]

    #If isPositive is not a list
    if isinstance(isPositive, int):
        isPositive = [isPositive]

    #Indicate that the targets are the paraphrased sentences
    targets = examples["sentence2"]

    #Tokenize the input
    modelInputs = tokenizer(inputs, max_length = 512, truncation = True, padding = "max_length")

    #Tokenize the output
    labels = tokenizer(targets, max_length = 512, truncation = True, padding = "max_length").input_ids

    #Ajust the outputs depending on the label (penalize the negatives)
    for i, elem in enumerate(isPositive):
        if elem == 0:
            labels[i] = [-100] + labels[i] #Penalize the output in the taining

    #Add the expected output to the input of the model
    modelInputs["labels"] = labels
    
    return modelInputs

##### Entrenamiento

In [9]:
savingPath = "/home/ibon/Documentos/1. Models/FlanT5Small"
modelName = "google/flan-t5-small"

#Possible characteristics of the models
dataDictList = [PAWSXDataDictMod]
freezePctgList = [0.5,0.8]
trainEpochsList = [6]

#Version control
version = 6

for i, dataDict in enumerate(dataDictList):
    for freezePctg in freezePctgList:
        for trainEpochs in trainEpochsList:
            trainFlanT5Model(savingPath, modelName, version, tokenizeFunctionFlanT5Mod, dataDict, aditionalTokens = None, trainEpochs = trainEpochs, freezePctg = freezePctg)

            with open(savingPath + "/VersionControlFlanT5Small.csv", "a") as file:
                data = "unlabeled \n" if i == 0 else "labeled \n"

                #Update the version control
                file.write(modelName.replace("/", "%") + "v" + str(version) + "," + str(version) + "," + str(freezePctg) + "," + str(trainEpochs) + "," + data)

            version += 1

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|██████████| 1180/1180 [00:00<00:00, 4294.11 examples/s]
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cach

Epoch,Training Loss,Validation Loss
1,0.234400,0.072752
2,0.198400,0.070854
3,0.214100,0.070051
4,0.183200,0.069201
5,0.220000,0.068537
6,0.202800,0.068647


Map: 100%|██████████| 1180/1180 [00:00<00:00, 4344.64 examples/s]
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.252100,0.075531
2,0.215400,0.073329
3,0.234800,0.072382
4,0.198900,0.071673
5,0.241800,0.071354
6,0.223500,0.071255


##### Probar los modelos

In [12]:
def generateTextFlanT5(modelPath, text):
    #Load the model
    model = T5ForConditionalGeneration.from_pretrained(modelPath)
    tokenizer = T5Tokenizer.from_pretrained(modelPath)

    #Check if the tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
        model.resize_token_embeddings(len(tokenizer))

    #Tokenize the input text
    inputIds = tokenizer(text, return_tensors = "pt").input_ids
    inputLength = inputIds.shape[-1]

    #Generate the text
    outputs = model.generate(inputIds, num_return_sequences = 5, max_length = inputLength, do_sample = True, num_beams = 5, early_stopping = True, repetition_penalty = 1.5, no_repeat_ngram_size = 2, top_k = 50, top_p = 0.95, temperature = 1.5,)

    return outputs

In [13]:
savingPath = "/home/ibon/Documentos/1. Models/FlanT5Small"
text = "Haz un parafraseo creativo de esta frase: Aunque la estación estás sucia, me gusta. Hay escaleras mecánicas por lo que es apta para inválidos"

#For every generated model
with open(savingPath + "/VersionControlFlanT5Small.csv", "r") as file:
    #Ignore the first line of the file
    next(file)

    for line in file:
        elemsList = line.split(",")
        
        modelName = elemsList[0]
        modelPath = savingPath + "/" + modelName

        #Generate the text
        outputs = generateTextFlanT5(modelPath, text)

        print("Modelo: " +  modelName + "\n")
        for output in outputs:
            #Decode the text
            generatedText = tokenizer.decode(output, skip_special_tokens=True)
            print("Texto: " + generatedText + "\n")

Modelo: google%flan-t5-smallv0

Texto: “Estación sucia, me gusta. Hay escaleras mecánicas por lo que está apta para inválidos.”

Texto: “Estación sucia, me gusta. Hay escaleras mecánicas por lo que está apta para inválidos”.

Texto: “Estación sucia, me gusta. Hay escaleras mecánicas por lo que está apta para inválidos.

Texto: “Estación sucia, me gusta. Hay escaleras mecánicas por lo que está apta para inválidos”

Texto: “Estación sucia, me gusta. Hay escaleras mecánicas por lo que está apta para inválidos

Modelo: google%flan-t5-smallv1

Texto: Aunque la estación está sucia, mi gusta. Hay escaleras mecánicas por lo que hay apta para inválidos.

Texto: Aunque la estación está sucia, mi gusta. Hay escaleras mecánicas por lo que hay apta para inválidos

Texto: Aunque la estación está sucia, mi gusta. Hay escaleras mecánicas por lo que hay apta para invisión.

Texto: Aunque la estación está sucia, mi gusta. Hay escaleras mecánicas por lo que hay apta para inválidos...

Texto: Aunque la es

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

sentence = "The train station is clean, but i do not like it. it does not have mechanical stairs"

text =  "paraphrase: " + sentence + " </s>"

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")

input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    do_sample=True,
    top_k=200,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=5
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(line)

The station is clean but i do not like it. It does not have mechanical stairs.
The railway station is clean but i do not like it because it does not have mechanical stairs.
The station is clean but i do not like it. It does not have mechanical stairs.
The train station is clean but i do not like it. It does not have mechanical stairs.
The station is clean, but i do not like it because it does not have mechanical stairs.


## DeepSeek R1

Debido a la imposibilidad de generar paráfrasis de calidad con los modelos GPT2 y T5, se optó por usar modelos de DeepSeek adaptados para la librería unsloth que permite reducir el consumo de vRAM durante el proceso de finetuning.

El modelo que se va a usar va a ser el unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit, que es una combinación de varias tecnologías desarrolladas por diferentes entidades, entre ellas DeepSeek y las técnicas de optimización de Unsloth.

### Funciones que se van a utilizar

Importar datos de dos ficheros (donde los datos no tienen etiquetas) a una lista de diccionarios de la forma: "sentence1" : original , "sentence2" : paráfrasis

In [3]:
def importParaphraseFromTxtToDictList(originalPath, paraphrasePath):
    dictList = []
    
    #As both files have the same number of lines, they can be iterated simultaneously
    with open(originalPath, 'r', encoding = 'utf-8') as originalFile, open(paraphrasePath, 'r', encoding = 'utf-8') as paraphraseFile:
        for originalLine, paraphraseLine in zip(originalFile, paraphraseFile): 
            dictList.append({"sentence1" : originalLine.strip(), "sentence2" : paraphraseLine.strip()})
            
    return dictList

Separación del conjunto de datos en: train, test y validation

In [4]:
def trainTestValidationSplit(dataset, trainPctg = 0.8):
    datasetSplit = dataset.train_test_split(test_size = 1 - trainPctg, seed = 54)
    validationTestSplit = datasetSplit["test"].train_test_split(test_size = 0.5, seed = 54)
    
    return DatasetDict({
        "train" : datasetSplit["train"],
        "test" : validationTestSplit["test"],
        "validation" : validationTestSplit["train"]
        })

Importación y filtrado de PAWSX (dataset de Google), de forma que se obtienen las frases en castellano y únicamente los datos que son paráfrasis (etiqueta a 1). Además, se van a eliminar las columnas de id y label, ya que no serán necesarias.

In [5]:
def processPAWSX():
    #Import the dataset with the spanish phrases
    datasetDict = load_dataset('paws-x', 'es')

    #Create an empty dataset to save the paraphrases
    newDatasetDict = DatasetDict()

    #Filter the paraphrases
    for key, valueDataset in datasetDict.items():
        paraphraseDataset = valueDataset.filter(lambda example: example['label'] == 1)

        paraphraseDataset = paraphraseDataset.remove_columns(["id", "label"])
    
        newDatasetDict[key] = paraphraseDataset
    
    return newDatasetDict

Formateo del promt para optimizar la tarea del modelo

In [6]:
def formattingPrompts(examples, instruction):
    inputs = examples["sentence1"]
    outputs = examples["sentence2"]
    texts = []
    for inpt, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = f"### Instrucción:\n{instruction}.\n\n### Entrada:\n{inpt}\n\n### Respuesta:\n{output}"+ EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

### Preparación de los datos

##### PAWSX filtrado
Dado que solo queremos hacer uso del modelo en castellano, únicamente nos quedaremos con las frases en este idioma. Además, el conjunto de datos original contiene pares de frases que son paráfrasis y que no lo son. En nuestro caso, únicamente nos interesan las primeras, por lo que también se filtrarán.

In [7]:
PAWSXDataDict = processPAWSX()

Estructura del dataset

In [8]:
print(PAWSXDataDict)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 21829
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 907
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 847
    })
})


Diez ejemplos no consecutivos para cerciorarnos de que todos los datos son paráfrasis

In [9]:
for i in range(10):
    print(PAWSXDataDict["train"][1050 + i * 2])

{'sentence1': 'El distrito electoral se encuentra en la bahía de Swansea, situada en la margen derecha del río Afan, cerca de su desembocadura en el sur de Gales.', 'sentence2': 'El distrito electoral se encuentra en Swansea Bay, en la orilla derecha del río Afan, cerca de su desembocadura en el sur de Gales.'}
{'sentence1': "Los `` Zapateros '' cayeron al 16º en 1991: 92, antes de caer en el 20º puesto en 1992: 93 bajo Phil Chard.", 'sentence2': "Los `` Zapateros '' cayeron al 16º en 1991: 92, antes de caer en el 20º puesto en 1992: 93 bajo Phil Chard."}
{'sentence1': 'La ley albanesa está codificada y basada en la ley francesa.', 'sentence2': 'La ley albanesa está codificada y se basa en la ley francesa.'}
{'sentence1': 'Nació en Usera, España (Madrid) el 18 de abril de 1976.', 'sentence2': 'Nació el 18 de abril de 1976 en Usera, España (Madrid).'}
{'sentence1': 'Los pequeños blenios marinos blenioides ("Ecsenius australianus") son peces australianos del género "Ecsenius".', 'sentenc

##### Dataset propio
Dado que hemos aplicado técnicas de aumento de datos, disponemos pares de frases (paráfrasis) adaptadas a nuestro dominio (reseñas de estaciones de metro). 

Se han aplicado 3 variaciones de la retrotraducción, por lo que solo con esto, se disponen de tres paraes de frases por cada elemento del conjunto.

In [10]:
invalidOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"
validOriginalPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"

backtranslationFolder = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/1. Back Translation/1. Google Translator"
invalidBacktranslationPath1 = backtranslationFolder + "/InvalidReviewsTranslationsEsEnEnEs.txt"
invalidBacktranslationPath2 = backtranslationFolder + "/InvalidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
invalidBacktranslationPath3 = backtranslationFolder + "/InvalidReviewsTranslationsEsJaJaEs.txt"
validBacktranslationPath1 = backtranslationFolder + "/ValidReviewsTranslationsEsEnEnEs.txt"
validBacktranslationPath2 = backtranslationFolder + "/ValidReviewsTranslationsEsFrFrJaJaRuRuEs.txt"
validBacktranslationPath3 = backtranslationFolder + "/ValidReviewsTranslationsEsJaJaEs.txt"

invalidParaphrasesDictList1 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath1)
invalidParaphrasesDictList2 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath2)
invalidParaphrasesDictList3 = importParaphraseFromTxtToDictList(invalidOriginalPath, invalidBacktranslationPath3)
validParaphrasesDictList1 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath1)
validParaphrasesDictList2 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath2)
validParaphrasesDictList3 = importParaphraseFromTxtToDictList(validOriginalPath, validBacktranslationPath3)

#Join all the lists
backtranslationDictList = invalidParaphrasesDictList1 + invalidParaphrasesDictList2 + invalidParaphrasesDictList3 + validParaphrasesDictList1 + validParaphrasesDictList2 + validParaphrasesDictList3
#Shuffle the lists
random.shuffle(backtranslationDictList)

#Convert it to a Hugging Face dataset
backtranslationDataset = Dataset.from_list(backtranslationDictList)

In [11]:
#Split the data
backtranslationDataDict = trainTestValidationSplit(backtranslationDataset)

In [12]:
print(backtranslationDataDict)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 7761
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 971
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 970
    })
})


##### Unión del PAWSX filtrado y el dataset propio

In [13]:
#Concatenate the datasets
unionDatasetDict = DatasetDict()

for key in PAWSXDataDict.keys():
    newDataset = concatenate_datasets([PAWSXDataDict[key], backtranslationDataDict[key]])

    unionDatasetDict[key] = newDataset

In [14]:
print(unionDatasetDict)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 29590
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 1878
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 1817
    })
})


### Diseño de los modelos

Se va a usar el paquete unsloth para cargar el modelo preentrenado poruqe permite una descarga y fine-tuning más rápidos

In [15]:
from unsloth import FastLanguageModel

modelName = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = modelName,
    max_seq_length = 2048, #Maximum lenght of the input sequence that the model can process
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.46.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.747 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Se añaden los adaptadores LoRA

In [16]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, #Rank of the low-rank matrices in LoRA adaptation
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0.05, #Introduces dropout to the low-rank matrices during the training of this LoRA adapter model
    bias = "none",    # Can be set to any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3977,
    use_rslora = False,  # unsloth also supports rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.12 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Diseño del promt

Un promt es una instrucción que se le da a un modelo de lenguaje para obtener una respuesta específica.

La ingeniería de promting es una técnica para diseñar promts de manera estratégica para obtener las mejores respuestas posibles.

Se va a añadir un campo a los datasets en el que se introduza el promt al completo (intrucción, frase original y frase parafraseada).

In [17]:
instruction = "Eres un experto en generación de datos sintéticos. Dado un texto de entrada, genera una paráfrasis que conserve el significado original pero utilice un vocabulario diferente y una estructura gramatical variada. El resultado debe ser útil para entrenar redes neuronales con datos sintéticos."

Dataset con solo los datos obtenidos de la técnica de retrotraducción

In [18]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
backtranslationDataDict = backtranslationDataDict.map(lambda x: formattingPrompts(x, instruction), batched = True,)

Map: 100%|██████████| 970/970 [00:00<00:00, 101722.04 examples/s]


Dataset con los datos de PAWS filtrado y los obtenidos mediante la retrotraducción

In [19]:
unionDatasetDict = unionDatasetDict.map(lambda x: formattingPrompts(x, instruction), batched = True,)

Map: 100%|██████████| 1817/1817 [00:00<00:00, 86339.90 examples/s]


In [20]:
unionDatasetDict

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'text'],
        num_rows: 29590
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'text'],
        num_rows: 1878
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'text'],
        num_rows: 1817
    })
})

### Entrenamiento de los modelos

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

max_seq_length = 2048

trainer = SFTTrainer(
    model = model, # The model with LoRA adapters
    tokenizer = tokenizer, # The tokenizer of the model
    train_dataset = unionDatasetDict["train"], # The dataset to use for training
    eval_dataset=unionDatasetDict["validation"],
    dataset_text_field = "text", # The field in the dataset that contains the structured data
    max_seq_length = max_seq_length, # Max length of input sequence that the model can process
    dataset_num_proc = 2, # Noe of processes to use for loading and processing the data
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # Batch size per GPU
        gradient_accumulation_steps = 4, # Step size of gradient accumulation
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 150, # Maximum steps of training
        learning_rate = 5e-5, # Initial learning rate
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # The optimizer that will be used for updating the weights
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        evaluation_strategy="steps",  # Evaluation by steps
        eval_steps=10,  # Evaluate every 10 steps
        save_strategy="steps",  # Save every 10 steps
        save_steps=10,  # Save every 10 steps
        load_best_model_at_end=True,  # Load the best model at the end
        metric_for_best_model="loss",  # Use the validation loss as the metric for the best model
    ),
)

/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Tokenizing eval dataset (num_proc=2): 100%|██████████| 1817/1817 [00:00<00:00, 6649.24 examples/s]
max_steps is given, it will override any value given in num_train_epochs


Entrenar y guardar el modelo

In [ ]:
trainer_stats = trainer.train()

version = 1

#Save the model
savingPath = "/home/ibon/Documentos/1. Models/DeepSeek-R1"
tokenizer.save_pretrained(savingPath + "/" + modelName.replace("/", "%") + "v" + str(version))
model.save_pretrained(savingPath + "/" + modelName.replace("/", "%") + "v" + str(version))

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 29,590 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 150
 "-____-"     Number of trainable parameters = 167,772,160
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'


Step,Training Loss,Validation Loss
10,0.469100,11.189466
20,0.411100,11.150325
30,0.531800,11.136603
40,0.521300,11.152895
50,0.444200,11.170336
60,0.555600,11.186615
70,0.534200,11.188937
80,0.962000,11.204674
90,0.561300,11.212711
100,0.537900,11.229830


'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
'UnslothSFTConfig' object has no attribute 'average_tokens_across_devices'
'UnslothSFTConfig' obj

In [27]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inpt = "El servicio es aceptable pero la etación es muy pequeña"
text = f"### Instrucción:\n{instruction}\n\n### Entrada:\n{inpt}\n\n### Respuesta:\n"
inputs = tokenizer(
[
    text
], return_tensors = "pt").to("cuda")



outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<｜begin▁of▁sentence｜>### Instrucción:\nEres un experto en generación de datos sintéticos. Dado un texto de entrada, genera una paráfrasis que conserve el significado original pero utilice un vocabulario diferente y una estructura gramatical variada. El resultado debe ser útil para entrenar redes neuronales con datos sintéticos.\n\n### Entrada:\nEl servicio es aceptable pero la etación es muy pequeña\n\n### Respuesta:\nEl servicio es aceptable, pero la estación es muy pequeña.<｜end▁of▁sentence｜>']

### Generación de paráfrasis

Cargar el modelo

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
savingPath = "/home/ibon/Documentos/1. Models/DeepSeek-R1"

#Path and version of the model
version = 1
modelPath = savingPath + "/" + modelName.replace("/", "%") + "v" + str(version)

#Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(modelPath)
model = FastLanguageModel.for_inference(modelPath) 

Generar las paráfrasis

In [ ]:
instruction = "Eres un experto en generación de datos sintéticos. Dado un texto de entrada, genera una paráfrasis que conserve el significado original pero utilice un vocabulario diferente y una estructura gramatical variada. El resultado debe ser útil para entrenar redes neuronales con datos sintéticos."

In [ ]:
inpt = "El servicio es aceptable pero la etación es muy pequeña"
text = f"### Instrucción:\n{instruction}\n\n### Entrada:\n{inpt}\n\n### Respuesta:\n"
inputs = tokenizer(
[
    text
], return_tensors = "pt").to("cuda")



outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True, temperature = 0.7)
tokenizer.batch_decode(outputs)

# Aumento de datos utilizando modelos de última generación vía API

Debido a las limitaciones hardware del equipo, la única solución viable para conseguir paráfrases de calidad usando redes neuronales es acceder a modelos vía API. Concretamente, se accederán a los modelos de OpenAI.

## Generación

### Imports necesarios

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv

### Cargar las variables de entorno

In [2]:
# Cargar variables de entorno desde .env
load_dotenv()

# Obtener la clave API
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

### Funciones que se van a utilizar

Función que hace una petición a la API de OpenAI

In [3]:
def generateText(model, systemMessageContent, prompt):
    systemMessage = {"role" : "system", "content" : systemMessageContent}
    userMessage = {"role" : "user", "content" : prompt}
    messages = [systemMessage, userMessage]

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )

    
    return response.choices[0].message.content

### Generación de las paráfrasis

#### Diseño del mensaje del sistema

Este mensaje del sistema está diseñado para asegurar que el modelo mantenga el significado original, use diferentes estructuras gramaticales y varíe el vocabulario sin perder claridad.

In [4]:
systemMessageContent = f"""Eres un experto en procesamiento del lenguaje natural con habilidades avanzadas en reformulación de textos.
Tu tarea es parafrasear un texto de manera que el significado se mantenga intacto, pero con cambios sustanciales en la estructura sintáctica y el vocabulario. 
Usa sinónimos, reformulaciones y construcciones alternativas para evitar repeticiones. 
Evita cambiar el tono y la intención del mensaje original. 
Si el texto contiene términos técnicos o nombres propios, consérvalos sin cambios.
No agregues ni elimines información que altere el significado original.
La respuesta debes proporcionarla en una linea"""

#### Diseño del prompt

El prompt está diseñado con la misma intención que el mensaje del sistema. 

In [5]:
def promptFunction(originalText):

    prompt = f""" Parafrasea el siguiente texto asegurando una alta variabilidad léxica y sintáctica, pero conservando el significado original:
    
    {originalText}
    
    Usa sinónimos y expresiones equivalentes.  
    Cambia la estructura de las oraciones sin perder claridad.  
    Mantén el tono y la intención del mensaje.  
    No agregues ni elimines información clave.  
    Asegúrate de que la reformulación sea natural y fluida. 
    Asegúrate de que la respuesta está toda en una linea.
    
    Respuesta: 
    """

    return prompt

#### Selección del modelo

Como se puede ver, con un modelo económico como gpt-4o-mini se obtinen resultados de muy alta calidad

In [6]:
model = "gpt-4o-mini"

text = "La estación es antigua, aparte de tener una sola salida de metro no está habilitada para personas con discapacidad. Además comunica por un pasillo subterráneo (que parece infinito) con Embajadores que pertenece a la línea 3 de metro"

parafrasis = generateText(model, systemMessageContent, promptFunction(text))

In [7]:
print(parafrasis)

La estación es vieja y, además de contar con una única entrada de metro, no está adaptada para personas con discapacidades; también conecta con Embajadores, que forma parte de la línea 3 de metro, a través de un pasillo subterráneo que parece no tener fin.


#### Parafrasis sobre el conjunto al completo

In [8]:
def generateParaphrasing(inputPath, outputPath):

    #Open input and output file
    with open(inputPath, "r", encoding = "utf-8") as inputFile, open(outputPath, "w", encoding = "utf-8") as outputFile:
        #Paraphrase every line in the input file
        for line in inputFile:
            
            line = line.strip()
            #Paraphrase
            paraphrase = generateText(model, systemMessageContent, promptFunction(line))
            #Write to the output file
            outputFile.write('"' + paraphrase + '"\n')

In [9]:
invalidInputPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"
validInputPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"

invalidOutputPath = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/2. Pretrained NLP/GPT4o/invalidParaphrasing.txt"
validOutputPath = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/2. Pretrained NLP/GPT4o/validParaphrasing.txt"

generateParaphrasing(invalidInputPath, invalidOutputPath)
generateParaphrasing(validInputPath, validOutputPath)


## Análisis de la generación

### Similitud Semántica

A continuación se va a diseñar una función para calcular la similitud semántica entre pares de oraciones. Es decir, se van a calcular los embeddings de oraciones de cada par de frases y se va a usar una métrica de similitud para ver como de parecido es el significado de ambas frases.

Se va a usar una versión de SBERT, llamada MiniLM (Minimal Lenguaje Model), que utiliza una variante más pequeña. Se usa MiniLM de seis capas (L6), que logra una precisón buena con menos recursos.

Este modelo fue entrenado usando un dataset que incluye datos en varios idiomas, entre ellos el español. Consecuentemente, no hay problema al introducir frases en castellano. Es cierto, que obtiene mejores resultados para frases en inglés, ya que se entreno con más datos en este idioma.

MiniLM es un modelo específicamente entrenado para mapear frases y parrafos a un espacio vectorial de 384 dimensiones. Es decir, este modelo permite obtener un embedding de una frase directamente. Usando otros modelos esta tarea no es posible de forma directa, ya que devuelven un embedding para cada palabra del texto.

El método de similitud que se va a usar es la similitud del coseno, por lo que los valores más cercanos a uno indicarán una mayor similitud entre las frases

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

#Given two texts and a model, the semantic similarity of the texts is returned
def getSemanticSimilarity(text1, text2, model):
    #Get the embeddings of the senteces
    embedding1 = model.encode(text1)
    embedding2 = model.encode(text2)

    #Get the cosine similarity of the senteces
    similarity = cosine_similarity([embedding1], [embedding2])

    return similarity[0][0]

### Similitud Léxica
Se va a diseñar una función para calcular la similitud léxica entre pares de oraciones. La similitud léxica mide el grado de coincidencia de palabras o términos entre dos frases o textos, sin tener en cuenta el significado subyacente.

Hay varias formas de realizar este cálculo: similitud del coseno basada en frecuencia de palabras, coeficiente de Jaccard,coeficiente de Dice ...

En este caso, se cree que la mejor opción es usar el coeficiente de Jaccard ya que calcula la similitud en función de la proporción de palabras comunes sobre el total de palabras únicas. Consecuentemente, esto nos permitirá detectar frases con menos coincidencias exactas en palabras.

Cuanto más cercano a uno sea el coeficiente de Jaccard más similares léxicamente serán las frases.

In [4]:
import unicodedata
import re

#Clean up the text removing punctuation, accent marks and convertin everything to lowercase
def cleanText(text):
    text = unicodedata.normalize('NFKD', text.lower()).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [5]:
def getSpanishStopWords():
    determinantes = {"el", "la", "los", "las", "un", "una", "unos", "unas", "este", "esta", "estos", "estas",
                 "ese", "esa", "esos", "esas", "aquel", "aquella", "aquellos", "aquellas", "mi", "mis",
                 "tu", "tus", "su", "sus", "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", 
                 "vuestra", "vuestros", "vuestras", "primer", "primero", "primera", "segundo", "segunda"}

    preposiciones = {"a", "ante", "bajo", "cabe", "con", "contra", "de", "desde", "durante", "en", "entre", 
                 "hacia", "hasta", "mediante", "para", "por", "según", "sin", "sobre", "tras", "versus", "vía"}

    conjunciones = {"y", "e", "ni", "o", "u", "pero", "sino", "sino que", "mas", "aunque", "que", "porque", 
                "como", "cuando", "donde", "mientras", "para que", "a fin de que", "puesto que", "ya que", 
                "si", "siempre que"}
    pronombres = {
        # Pronombres personales
        "yo", "tú", "vos", "él", "ella", "nosotros", "nosotras", 
        "vosotros", "vosotras", "ellos", "ellas", "usted", "ustedes",
        "me", "te", "lo", "la", "nos", "os", "los", "las", "le", "les", "se",
    
        # Pronombres posesivos
        "mío", "mía", "míos", "mías", 
        "tuyo", "tuya", "tuyos", "tuyas", 
        "suyo", "suya", "suyos", "suyas", 
        "nuestro", "nuestra", "nuestros", "nuestras", 
        "vuestro", "vuestra", "vuestros", "vuestras",
    
        # Pronombres demostrativos
        "este", "esta", "estos", "estas", 
        "ese", "esa", "esos", "esas", 
        "aquel", "aquella", "aquellos", "aquellas",
    
        # Pronombres relativos
        "que", "cual", "cuales", "quien", "quienes", 
        "cuyo", "cuya", "cuyos", "cuyas", "donde",
    
        # Pronombres interrogativos y exclamativos
        "qué", "quién", "quiénes", "cuál", "cuáles", 
        "cuánto", "cuánta", "cuántos", "cuántas", 
        "dónde", "cómo", "cuándo",
    
        # Pronombres indefinidos
        "alguien", "algo", "nadie", "nada", "cualquiera", 
        "todos", "todas", "varios", "varias", "muchos", 
        "muchas", "pocos", "pocas", "alguno", "alguna", 
        "algunos", "algunas", "ninguno", "ninguna", 
        "uno", "una", "unos", "unas", "demás"
    }

    #Combine all the words in one set
    spanishStopWords = determinantes | preposiciones | conjunciones | pronombres

    return spanishStopWords

In [6]:
def revomeSpanishStopWords(text):
    spanishStopWords = getSpanishStopWords()

    textWithoutStopWords = [word for word in text.split() if word.lower() not in spanishStopWords]

    return " ".join(textWithoutStopWords)

In [7]:
#Jaccard similarity
def jaccardSimilarity(text1, text2):
    #Get the set of words of each text
    wordsInText1 = set(revomeSpanishStopWords(cleanText(text1)).split())
    wordsInText2 = set(revomeSpanishStopWords(cleanText(text2)).split())

    intersection = len(wordsInText1.intersection(wordsInText2)) 
    union = len(wordsInText1.union(wordsInText2))
    
    if union == 0:
        return 0

    #intersection / union
    return intersection / union

In [8]:
#Given two texts, the Jaccard similarity of those texts is returned
def getLexicalSimilarity(text1, text2):
    return jaccardSimilarity(text1, text2)

### Similitudes

In [9]:
from sentence_transformers import SentenceTransformer

#originalDataList: list of texts representing the original dataset
#allAugmentedDataList: list of list of texts representing the aumented data 
#(allAugmentedDataList = [augmentedDataList1, ... ,augmentedDataListN], where augmentedDataList = [augmentedData1, ..., augmentedDataM])
#pathWithOriginal: path of the csv with two columns (the original text and the best augmented text)
#pathAugmentedData: path of the file with only the augmented data (without the original text)
def processAugmentation(originalDataList, allAugmentedDataList, pathWithOriginal, pathAugmentedData):
    #Select the model for the semantic similarity
    model = SentenceTransformer('all-MiniLM-L6-v2')

    #Open the files in which the augmented data will be strored
    withOriginalFile = open(pathWithOriginal, "w", encoding="utf-8")
    augmentedDataFile = open(pathAugmentedData, "w", encoding="utf-8")

    #Write the titles of the csv
    withOriginalFile.write("OriginalText,AugmentedText,SemanticSimilarity,LexicalSimilarity\n")
    
    resul = []
    #Analize every phrase in the original data
    for i, originalText in enumerate(originalDataList):
        allAugmentedDataInfoDict = {}
        bestIdx = 1
        
        #Analize every traduction
        for j, augmentedDataList in enumerate(allAugmentedDataList):
            #Compute the similarities of the corresponding traduction
            semanticSimilarity = getSemanticSimilarity(originalText, augmentedDataList[i], model)
            lexicalSimilarity = getLexicalSimilarity(originalText, augmentedDataList[i])

            #Save the traduction and the similarities in a dictionary
            allAugmentedDataInfoDict.update({
                f"augmented{j + 1}": augmentedDataList[i],
                f"semanticSimilarity{j + 1}": semanticSimilarity,
                f"lexicalSimilarity{j + 1}": lexicalSimilarity
            })

            #Get the index of the traduction with greater semantic similarity and less lexical similarity
            bestIdx = max(bestIdx, j + 1,
                key = lambda k: (allAugmentedDataInfoDict[f"semanticSimilarity{k}"] - allAugmentedDataInfoDict[f"lexicalSimilarity{k}"])
            )

        #Select the information of the best augmentation
        info = {
            "originalText": originalText,
            "bestAugmentation": allAugmentedDataInfoDict[f"augmented{bestIdx}"],
            "bestAugmentedDataSemanticSimilarity": allAugmentedDataInfoDict[f"semanticSimilarity{bestIdx}"],
            "bestAugmentedDataLexicalSimiliratity": allAugmentedDataInfoDict[f"lexicalSimilarity{bestIdx}"]
        }
        info.update(allAugmentedDataInfoDict)

        #Save the information
        resul.append(info)

        #Write the information in the files
        withOriginalFile.write(originalText + "," + allAugmentedDataInfoDict[f"augmented{bestIdx}"] + "," + str(allAugmentedDataInfoDict[f"semanticSimilarity{bestIdx}"]) + "," + str(allAugmentedDataInfoDict[f"lexicalSimilarity{bestIdx}"]) + "\n")
        #If the text is not empty write it on  the file
        if allAugmentedDataInfoDict[f"augmented{bestIdx}"]  != '""':
            augmentedDataFile.write(allAugmentedDataInfoDict[f"augmented{bestIdx}"] + "\n")

    #Close the files
    withOriginalFile.close()
    augmentedDataFile.close()
    
    return resul

/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2025-03-01 17:35:01.707689: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 17:35:01.715174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740846901.725243    6676 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1

In [10]:
def importFromTxtToList(source):
    with open(source, 'r', encoding="utf-8") as file:
        #Generate a list with all the reviews
        targetList = [line.strip() for line in file]
    return targetList

In [11]:
invalidPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/InvalidReviews.txt"
validPath = "/home/ibon/Documentos/GitHub/TFG/1. Data/4. Labeled Reviews/2. Without Emojis/ValidReviews.txt"

invalidList = importFromTxtToList(invalidPath)
validList = importFromTxtToList(validPath)

invalidParaphrasedPath = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/2. Pretrained NLP/GPT4oMini/invalidParaphrasing.txt"
validParaphrasedPath = "/home/ibon/Documentos/GitHub/TFG/2. Review Classifier/1. Data Augmentation/2. Pretrained NLP/GPT4oMini/validParaphrasing.txt"

invalidParaphrasedList = importFromTxtToList(invalidParaphrasedPath)
validParaphrasedList = importFromTxtToList(validParaphrasedPath)

In [12]:
validWithOriginalPath = '2. Pretrained NLP/2. Augmented Data/ValidGPT4oMiniWithOriginal.csv'
validAugmentedPath = '2. Pretrained NLP/2. Augmented Data/ValidGPT4oMiniData.txt'
invalidWithOriginalPath = '2. Pretrained NLP/2. Augmented Data/InvalidGPT4oMiniWithOriginal.csv'
invalidAugmentedPath = '2. Pretrained NLP/2. Augmented Data/InvalidGPT4oMiniData.txt'

infoValid = processAugmentation(validList, [validParaphrasedList], validWithOriginalPath, validAugmentedPath)
infoValidDF = pd.DataFrame(infoValid)
infoInvalid = processAugmentation(invalidList, [invalidParaphrasedList], invalidWithOriginalPath, invalidAugmentedPath)
infoInvalidDF = pd.DataFrame(infoInvalid)

In [15]:
infoValidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1
0,"""Tiene fácil acceso para las personas con movi...","""El lugar es fácilmente accesible para individ...",0.791221,0.280000,"""El lugar es fácilmente accesible para individ...",0.791221,0.280000
1,"""Espero que hayan mejorais""","""""Confío en que hayan progresado.""""",0.547604,0.200000,"""""Confío en que hayan progresado.""""",0.547604,0.200000
2,"""La estación es antigua, aparte de tener una s...","""La estación es vetusta y, además de tener úni...",0.844201,0.424242,"""La estación es vetusta y, además de tener úni...",0.844201,0.424242
3,"""Bien""","""""Correcto""""",0.370179,0.000000,"""""Correcto""""",0.370179,0.000000
4,"""Bonito comodo""","""""Agradable y confortable.""""",0.202906,0.000000,"""""Agradable y confortable.""""",0.202906,0.000000


In [18]:
infoInvalidDF.head()

,originalText,bestAugmentation,bestAugmentedDataSemanticSimilarity,bestAugmentedDataLexicalSimiliratity,augmented1,semanticSimilarity1,lexicalSimilarity1
0,"""He vivido 35 años en el barrio y reconozco qu...","""""Después de haber pasado 35 años en este veci...",0.815581,0.289474,"""""Después de haber pasado 35 años en este veci...",0.815581,0.289474
1,"""localización con muchos bares interesantes""","""""zona con una gran cantidad de bares atractiv...",0.633096,0.142857,"""""zona con una gran cantidad de bares atractiv...",0.633096,0.142857
2,"""…""","""Parece que no has proporcionado un texto para...",0.280626,0.000000,"""Parece que no has proporcionado un texto para...",0.280626,0.000000
3,"""Muy rica comida..""","""""Deliciosa gastronomía.""""",0.309440,0.000000,"""""Deliciosa gastronomía.""""",0.309440,0.000000
4,"""Estación del.metro""","""""Terminal del tren subterráneo""""",0.342315,0.000000,"""""Terminal del tren subterráneo""""",0.342315,0.000000
